In [ ]:
from ase.io import read, write
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core import Lattice, Structure, Molecule
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core import PeriodicSite

from tqdm import tqdm
from pymatgen.core import Composition, Structure
from typing import List, Tuple

import re, joblib, json
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from pyxtal.symmetry import Group
import pickle
import pandas as pd
from ase.io import read, write
from ase import Atoms
from ase.data import chemical_symbols
from ase.db import connect
import numpy as np
import sys, os
from spglib import get_spacegroup, find_primitive, standardize_cell
import pymatgen 
from pymatgen.io.ase import AseAtomsAdaptor
import ast
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import roc_curve, auc,accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score # for cross-validation
from datetime import datetime 
from sklearn.preprocessing import LabelEncoder,OneHotEncoder, StandardScaler
from pymatgen.core import Composition, Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.io.cif import CifParser
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pymatgen.io.cif import CifWriter
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
import math
from tqdm import tqdm
from sklearn.neighbors import KernelDensity
from dis_gen.model import CrystalDataset, VAE
from dis_gen.generation import decode_samples, decode_samples_xy
from dis_gen.csp_filter import structure_validity,oxidation_state_validity,Symmetry_matching_accuracy
from collections import defaultdict
import pyxtal as px
from pyxtal.tolerance import Tol_matrix
from pyxtal.symmetry import search_cloest_wp, Group
from pymatgen.io.cif import CifParser
from io import StringIO

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super().default(obj)


2025-07-22 09:47:32.562380: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-22 09:47:32.568883: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-22 09:47:32.604592: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-22 09:47:32.604637: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-22 09:47:32.604668: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
SG_SYM = {spacegroup: Group(spacegroup) for spacegroup in range(1, 231)}
SG_TO_WP_TO_SITE_SYMM = dict()
for spacegroup in range(1, 231):
    SG_TO_WP_TO_SITE_SYMM[spacegroup] = dict()
    for wp in SG_SYM[spacegroup].Wyckoff_positions:
        wp_site = str(wp.multiplicity)+wp.letter
        wp.get_site_symmetry()
        SG_TO_WP_TO_SITE_SYMM[spacegroup][wp_site] = wp
#       SG_TO_WP_TO_SITE_SYMM[spacegroup][wp_site][wp] = wp.get_site_symmetry_object().to_one_hot()

In [6]:
SG_TO_WP_TO_SITE_SYMM[139].keys()

dict_keys(['32o', '16n', '16m', '16l', '16k', '8j', '8i', '8h', '8g', '8f', '4e', '4d', '4c', '2b', '2a'])

In [2]:
# Load the VAE model
model_dir = 'New_Kl5_ICSD_dis_site_middle_KL_element1000_lr_5e-06_epochs_2500_batch_64_test_0.2_val_0.1'
best_model = True

if best_model:
    vae_dict = torch.load(model_dir+'/best_vae_model.pth',map_location=torch.device('cpu'))
else: # use exit model
    vae_dict = torch.load(model_dir+'/exit_vae_model.pth',map_location=torch.device('cpu'))

feature_dim = 183
wyckoff_dim = 9
crystal_dim = 236
kernel = [5,3,2]
stride = [2,3,2]
max_filter = 16
latent_dim = 256
vae_eval = VAE(feature_dim, wyckoff_dim, crystal_dim,verbose=False,kernel=kernel,stride=stride)
vae_eval.load_state_dict(vae_dict['model'])

# Decode the latent space samples
latent_space_path = 'z_sample_NewICSD_test.npy'
z_samples = np.load(latent_space_path)

if 'Y_scaler' in vae_dict:
    scaler_Y = vae_dict['Y_scaler']
else:
    # Load the scaler if not present in the model dictionary
    print('Loading Y_scaler from file Y_scaler.gz')
    scaler_Y = joblib.load('Y_scaler.gz')

save_dict = decode_samples(z_samples,vae_eval,scaler_Y,device='cpu')

# Save the decoded samples to a file
#output_file = f'Decoded_data/decoded_{latent_space_path.split("/")[-1].replace(".npy", ".json")}'

#with open(output_file, 'w') as f:
#    json.dump(save_dict, f, cls=NumpyEncoder)
    
#print(f'Decoded samples saved to {output_file}')


Loading Y_scaler from file Y_scaler.gz


/home/energy/mahpe/Published_code/Dis-CSP/dis_csp/generation.py:148: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'disordered_site': F.sigmoid(torch.tensor(disordered_site)).cpu().detach().numpy(),


### Do not run, loading of data and saved to latent space

In [3]:
X = np.load('/home/energy/mahpe/Published_code/Dis-gen/New_atomic_features_icsd_all_new.npy')
Y = np.load('/home/energy/mahpe/Published_code/Dis-gen/New_crystal_features_icsd_all_new.npy')
print('X:',X.shape)
print('Y:',Y.shape)
test_size = 0.2
val_size = 0.1
save = False
# Setup scaler for Y
scaler_Y = StandardScaler()
Y_scaled = Y.copy()
Y_scaled[:,:6] = scaler_Y.fit_transform(Y[:,:6])
# Set X and Y to torch tensors
Y = Y_scaled

x_train_val,x_test,y_train_val,y_test=train_test_split( X \
                                               ,Y,test_size=test_size,random_state=42)
x_train,x_val,y_train,y_val = train_test_split(x_train_val,y_train_val,test_size=val_size,random_state=42)

print('x_train:',x_train.shape)
print('x_val:',x_val.shape)
print('x_test:',x_test.shape)
if save:
    np.save('/home/energy/mahpe/Published_code/dis_gen_X_train.npy', x_train)
    np.save('/home/energy/mahpe/Published_code/dis_gen_X_val.npy', x_val)
    np.save('/home/energy/mahpe/Published_code/dis_gen_X_test.npy', x_test)

    np.save('/home/energy/mahpe/Published_code/dis_gen_Y_train.npy', y_train)
    np.save('/home/energy/mahpe/Published_code/dis_gen_Y_val.npy', y_val)
    np.save('/home/energy/mahpe/Published_code/dis_gen_Y_test.npy', y_test)

    x_test = torch.tensor(x_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32)
    z_mean_test, z_var_test = vae_eval.encode(x_test,y_test)
    z_sample_test = vae_eval.sampling(z_mean_test, z_var_test)
    np.save('z_sample_NewICSD_test.npy', z_sample_test.cpu().detach().numpy())

    x_val = torch.tensor(x_val, dtype=torch.float32)
    y_val = torch.tensor(y_val, dtype=torch.float32)
    z_mean_val, z_var_val = vae_eval.encode(x_val,y_val)
    z_sample_val = vae_eval.sampling(z_mean_val, z_var_val)
    np.save('z_sample_NewICSD_val.npy', z_sample_val.cpu().detach().numpy())

    x_train = torch.tensor(x_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32)
    z_mean_train, z_var_train = vae_eval.encode(x_train,y_train)
    z_sample_train = vae_eval.sampling(z_mean_train, z_var_train)
    np.save('z_sample_NewICSD_train.npy', z_sample_train.cpu().detach().numpy())

X: (135038, 9, 183)
Y: (135038, 236)
x_train: (97227, 9, 183)
x_val: (10803, 9, 183)
x_test: (27008, 9, 183)


In [5]:
df = pd.read_csv('/home/energy/mahpe/Published_code/Dis-gen/ICSD2024_summary_2024.2_v5.3.0_ascending.csv')
quary_id = np.load('/home/energy/mahpe/Published_code/Dis-gen/New_quary_id_icsd_all_sorted.npy')
df_sort = df.loc[df['QueryID'].isin(quary_id)]
print('df_sort:',df_sort.shape)
test_size = 0.2
val_size = 0.1
quary_id_train_val,quary_id_test=train_test_split( quary_id,test_size=test_size,random_state=42)
quary_id_train, quary_id_val = train_test_split(quary_id_train_val, test_size=val_size, random_state=42)
print('quary_id_test:',len(quary_id_test))
print('quary_id_train:',len(quary_id_train))
print('quary_id_val:',len(quary_id_val))

/tmp/ipykernel_2877215/2755769295.py:1: DtypeWarning: Columns (22,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/home/energy/mahpe/Published_code/Dis-gen/ICSD2024_summary_2024.2_v5.3.0_ascending.csv')


df_sort: (135038, 32)
quary_id_test: 27008
quary_id_train: 97227
quary_id_val: 10803


In [15]:
## Save ICSD ids
list_df_train = []
for i in range(len(quary_id_train)):
    list_df_train.append( df[df['QueryID']==quary_id_train[i]].iloc[0].to_dict())

df_train = pd.DataFrame(list_df_train)
icsd_id_train = df_train['CollectionCode'].values

list_df_val = []
for i in range(len(quary_id_val)):
    list_df_val.append( df[df['QueryID']==quary_id_val[i]].iloc[0].to_dict())

df_val = pd.DataFrame(list_df_val)
icsd_id_val = df_val['CollectionCode'].values

list_df_test = []
for i in range(len(quary_id_test)):
    list_df_test.append( df[df['QueryID']==quary_id_test[i]].iloc[0].to_dict())

df_test = pd.DataFrame(list_df_test)
icsd_id_test = df_test['CollectionCode'].values

print('icsd_id_test:',len(icsd_id_test))
print('icsd_id_train:',len(icsd_id_train))
print('icsd_id_val:',len(icsd_id_val))

# Save the ICSD ids
np.savetxt('data_ICSD/dis_gen_icsd_id_train.txt', icsd_id_train, fmt='%s')
np.savetxt('data_ICSD/dis_gen_icsd_id_val.txt', icsd_id_val, fmt='%s')
np.savetxt('data_ICSD/dis_gen_icsd_id_test.txt', icsd_id_test, fmt='%s')

icsd_id_test: 27008
icsd_id_train: 97227
icsd_id_val: 10803


In [4]:
X_test = np.load('/home/energy/mahpe/Published_code/dis_gen_X_test.npy')
Y_test_notscaled = np.load('/home/energy/mahpe/Published_code/dis_gen_Y_test.npy')
Y_test = Y_test_notscaled.copy()

decode_dict = decode_samples_xy(X_test, Y_test, scaler_Y, device='cpu')
abc = decode_dict['abc']
angles = decode_dict['angles']
spg = decode_dict['spacegroup']

list_df = []
for i in range(len(quary_id_test)):
    list_df.append( df[df['QueryID']==quary_id_test[i]].iloc[0].to_dict())

df_test = pd.DataFrame(list_df)
cell_parameters = df_test['CellParameter'].values

import re
pattern = r"([\d.]+)(?:\((\d+)\))?"
abc_list = []
angles_list = []
for cell in cell_parameters:
    matches = re.findall(pattern, cell)
    abc_i = [float(match[0]) for match in matches[:3]]
    angles_i = [float(match[0]) for match in matches[3:6]]
    abc_list.append(abc_i)
    angles_list.append(angles_i)
abc_array = np.array(abc_list)
angles_array = np.array(angles_list)
print(np.mean(abc -abc_array), np.mean(angles - angles_array))

mae_a = mae(abc[:,0], save_dict['abc'][:,0])
mae_b = mae(abc[:,1], save_dict['abc'][:,1])
mae_c = mae(abc[:,2], save_dict['abc'][:,2])
mae_alpha = mae(angles[:,0], save_dict['angles'][:,0])
mae_beta = mae(angles[:,1], save_dict['angles'][:,1])
mae_gamma = mae(angles[:,2], save_dict['angles'][:,2])
print('MAE abc:', mae_a, mae_b, mae_c)
print('MAE angles:', mae_alpha, mae_beta, mae_gamma)

spg_accuracy = accuracy_score(spg, save_dict['spacegroup'])
print('Spacegroup accuracy:', spg_accuracy)

NameError: name 'quary_id_test' is not defined

In [10]:
def align_fractional_coordinates(fractional_coord,wp_site,spg,spg_group):
    """
    Align the fractional coordinates to the Wyckoff position.
    
    Args:
        fractional_coord (np.ndarray): The fractional coordinates to align.
        wp_site (str): The Wyckoff position site (e.g., '2a', '3b').
        spg (int): The space group number.
        spg_group (pyxtal.symmetry.Group): The space group object.
    
    Returns:
        np.ndarray: The aligned fractional coordinates.
        pyxtal.symmetry.WyckoffPosition: The Wyckoff position object.
        int: The orbit index of the Wyckoff position.
    """
    wp = SG_TO_WP_TO_SITE_SYMM[spg][wp_site]

    closes =[]
    for orbit_index in range(len(wp.ops)):
        close = search_cloest_wp(spg_group, wp, wp.ops[orbit_index], fractional_coord)
        closes.append((close, wp, orbit_index, np.linalg.norm(np.minimum((close - fractional_coord)%1., (fractional_coord - close)%1.)) ))

    # Get the closest fractional coordinates
    closest = sorted(closes, key=lambda x: x[-1])[0]
    return closest[0], closest[1], closest[2]  # Return the closest fractional coordinates, Wyckoff position, and orbit index 


def generate_wyckoffgene(data_pkl, max_iter = None,element_acc = 0.01,disorder_acc = 0.1,shift_frac_coord=True,verbose = False):
    """
    Process the data from a dictionary containing crystal structure information and return a list of symmetrized structures.
    Args:
        data_pkl (dict): Dictionary containing crystal structure data with keys 'abc', 'angles', 'spacegroup', 'element', 'wyckoff_letter', 'wyckoff_mult', 'frac_coords', and 'disordered_site'.
        max_iter (int, optional): Maximum number of structures to process. If None, all structures are processed.
        element_acc (float, optional): Minimum fraction for an element to be considered present.
        disorder_acc (float, optional): Threshold for determining if a site is disordered.
        shift_frac_coord (bool, optional): If True, shift fractional coordinates to align with Wyckoff positions.
        verbose (bool, optional): If True, print additional information during processing.
    Returns:
        list: A list of wyckoffgene dictionaries, each containing information about a crystal structure.
        int: Number of symmetry matching accuracy failures.
        int: Number of failed items due to invalid lattice parameters or other issues.
    """

    wyckoffgenes_list = [] 
    failed_count = 0  # Counter for failed items
    SMA_count = 0  # Counter for symmetry matching accuracy failures
    for i in tqdm(range(len(data_pkl['abc']))):
        # # Create the structure            
        spacegroup_int = data_pkl['spacegroup'][i]
        spacegroup_group = SG_SYM[spacegroup_int] if spacegroup_int in SG_SYM else Group(spacegroup_int)     
        
        # Define the lattice parameters
        lattice = Lattice.from_parameters(
            a=data_pkl['abc'][i][0],
            b=data_pkl['abc'][i][1],
            c=data_pkl['abc'][i][2],
            alpha=data_pkl['angles'][i][0],
            beta=data_pkl['angles'][i][1],
            gamma=data_pkl['angles'][i][2]
        )
        # Check if the lattice is valid
        if any(np.array(lattice.abc) == 0):
            if verbose:
                print(f"Invalid lattice parameters for index {i}: {lattice.abc}")
            failed_count += 1
            continue
        # create lattic object
        # Identify if a Wyckoff site is empty
        index_max = np.argmax(data_pkl['wyckoff_letter'][i], axis=1)
        index = index_max != 0

        # Define if the site is disordered
        disordered_site = data_pkl['disordered_site'][i][index]
        disordered_site = disordered_site>disorder_acc

        # Loop trough all wyckoff sites and define the element
        index_array = np.arange(0,len(index))[index]
        element_comb = []
        for site, a_list in enumerate(data_pkl['element'][i][index]):

            if disordered_site[site]: # Disordered
                element_index = np.where(a_list > element_acc)[0] + 1
                disordered = True
            else: # Ordered
                element_index = np.argmax(a_list) + 1
                disordered = False                    
            
            if disordered:
                disordered_element = {chemical_symbols[elem]: float(data_pkl['element'][i][site][elem - 1]) for elem in element_index if chemical_symbols[elem] !='Md'} # Md is vacancies
                if len(disordered_element) == 0:  # If no elements are present, skip this site
                    index[index_array[site]] = False  # Mark the site as not present
                else:
                    element_comb.append(disordered_element)
            else:
                if chemical_symbols[element_index] == 'Md':  # If the element is Md, it is a vacancy
                    index[index_array[site]] = False  # Mark the site as not present
                else:
                    element_comb.append({chemical_symbols[element_index]: 1.0})
        
        # Define the Fractional coordinates
        frac_coords = data_pkl['frac_coords'][i][index]
        #print(element_comb)
        # Define the Wyckoff sites letters and multipliers            
        w_letter = np.array([chr(ord('a') + l - 1) for l in np.argmax(data_pkl['wyckoff_letter'][i][index], axis=1) if l != 0])
        w_multiplier = np.array([m for m in np.argmax(data_pkl['wyckoff_mult'][i][index], axis=1) if m != 0])
        w_site = [str(w) + l for w, l in zip(w_multiplier, w_letter)]

        # Check if the length of Wyckoff letters and multipliers match. A mismatch can happen if either the letter or multiplier is zero.
        if len(w_letter) != len(w_multiplier):
            if verbose:
                print(index)
                print(w_letter, w_multiplier,w_site)
                print(f"Mismatch in length of Wyckoff letters and multipliers for index {i}.")
            failed_count += 1
            continue
        
        # Check if the symmetry matching is correct
        symm_acc = Symmetry_matching_accuracy(w_letter, w_multiplier, spacegroup_group)
        if not symm_acc:
            if verbose:
                print(f"Symmetry accuracy failed for index {i} with spacegroup {spacegroup_int} and Wyckoff sites {w_site}")
            SMA_count += 1
            continue
        if verbose:
            print(f"Symmetry accuracy passed for index {i} with spacegroup {spacegroup_int} and Wyckoff sites {w_site}")

        # Shift fractional coordinates if needed 
        if shift_frac_coord:
            coord_list = []
            for wp_site, coord in zip(w_site, frac_coords):

                # Extra check for symmetry accuracy
                if wp_site not in SG_TO_WP_TO_SITE_SYMM[spacegroup_int].keys():
                    if verbose:
                        print(f"Wyckoff site {wp_site} not found in spacegroup {spacegroup_int}.")
                    SMA_count += 1
                    symm_acc = False
                    break 

                # Align the fractional coordinates to the Wyckoff position
                coord, _, _ = align_fractional_coordinates(coord, wp_site, spacegroup_int, spacegroup_group)
                coord_list.append(coord)
            if not symm_acc:
                continue 

            frac_coords = coord_list
        
        # Get the wyckoffgene 
        element_wyckoff = defaultdict(list)
        element_counts = defaultdict(int)
        elements_occ = defaultdict(lambda: defaultdict(list))
        elements_frac_coord = defaultdict(lambda: defaultdict(list))

        for el_dict, wyck, fr in zip(element_comb, w_site, frac_coords):
            for el, occ in el_dict.items():
                element_wyckoff[el].append(wyck)
                element_counts[el] += int(round(int(''.join(filter(str.isdigit, wyck)))))  # assume multiplicity from Wyckoff symbol
                elements_occ[el][wyck].append(occ)  # accumulate occupancy for each element and wyckoff site
                elements_frac_coord[el][wyck].append(fr)  # accumulate fractional coordinates for each elemen        

        # Step 2: Build wyckoffgene
        wyckoffgene = {
            'spacegroup': spacegroup_int,
            'species': list(element_wyckoff.keys()),
            'sites': list(element_wyckoff.values()),
            'numIons': list(element_counts.values()),
            'lattice':px.lattice.Lattice(ltype=spacegroup_group.lattice_type,
                        volume=lattice.volume,
                        matrix=lattice.matrix
                        ),
            'occupancy': elements_occ,  # Assuming full occupancy for simplicity    
            'frac_coord': elements_frac_coord,
            'structure_pymatgen': Structure(lattice, element_comb, frac_coords),
            'spacegroup_group':spacegroup_group,
        }

        wyckoffgenes_list.append(wyckoffgene)
        if max_iter is not None and i >= max_iter:
            break
    return wyckoffgenes_list, SMA_count, failed_count
# Generate Wyckoff genes
mixiter = None
element_acc = 0.0001  
disorder_acc = 0.5
all_wyckoffgenes, sma_int, failed_int = generate_wyckoffgene(decode_dict,
                             max_iter=mixiter,
                             element_acc=element_acc,
                             disorder_acc=disorder_acc,
                             shift_frac_coord=True,
                             verbose=False)
if mixiter is not None:
    print(f"Symmetry Matching Accuracy: {100-sma_int/mixiter*100:.2f}%")
    print(f"Number of failed items: {failed_int} of {mixiter} processed, Procentage: {failed_int/mixiter*100:.2f}%")
else:
    print(f"Symmetry Matching Accuracy: {100- sma_int/len(save_dict['spacegroup'])*100:.2f}%")
    print(f"Number of failed items: {failed_int} of {len(save_dict['spacegroup'])} processed, Procentage: {failed_int/len(save_dict['spacegroup'])*100:.2f}%")


100%|██████████| 27008/27008 [1:06:23<00:00,  6.78it/s]

Symmetry Matching Accuracy: 100.00%
Number of failed items: 0 of 27008 processed, Procentage: 0.00%


In [39]:
all_wyckoffgenes[0]['structure_pymatgen'],all_wyckoffgenes[0]['structure_pymatgen'].distance_matrix.shape

(Structure Summary
 Lattice
     abc : 3.9 3.9 12.1
  angles : 90.0 90.0 90.0
  volume : 184.041
       A : 3.9 0.0 2.3880612583373386e-16
       B : -2.3880612583373386e-16 3.9 2.3880612583373386e-16
       C : 0.0 0.0 12.1
     pbc : True True True
 PeriodicSite: Ce:0.085, Nd:0.915 (1.95, 1.95, 1.787) [0.5, 0.5, 0.1477]
 PeriodicSite: Cu (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
 PeriodicSite: O (1.95, 0.0, 1.194e-16) [0.5, 0.0, 0.0]
 PeriodicSite: O (-1.194e-16, 1.95, 3.025) [0.0, 0.5, 0.25],
 (4, 4))

In [48]:
cif_files_list = [get_cif_lines(wyckoffgene) for wyckoffgene in all_wyckoffgenes]

In [50]:
np.save('/home/energy/mahpe/Published_code/dis_gen_all_cifs_test.npy', np.array(cif_files_list))

In [ ]:
from dis_gen.generation import get_cif_lines
cif_lines = get_cif_lines(all_wyckoffgenes[0])
structure = Structure.from_str(cif_lines, fmt='cif')
structure, structure.distance_matrix.shape

/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/pymatgen/io/cif.py:1229: UserWarning: Issues encountered while parsing CIF: Skipping relative stoichiometry check because CIF does not contain formula keys.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


(Structure Summary
 Lattice
     abc : 3.9 3.9 12.1
  angles : 90.0 90.0 90.0
  volume : 184.041
       A : 3.9 0.0 2.3880612583373386e-16
       B : -2.3880612583373386e-16 3.9 2.3880612583373386e-16
       C : 0.0 0.0 12.1
     pbc : True True True
 PeriodicSite: Nd (Ce:0.085, Nd:0.915) (1.95, 1.95, 1.791) [0.5, 0.5, 0.148]
 PeriodicSite: Nd (Ce:0.085, Nd:0.915) (1.95, 1.95, 10.31) [0.5, 0.5, 0.852]
 PeriodicSite: Nd (Ce:0.085, Nd:0.915) (0.0, 0.0, 7.841) [0.0, 0.0, 0.648]
 PeriodicSite: Nd (Ce:0.085, Nd:0.915) (0.0, 0.0, 4.259) [0.0, 0.0, 0.352]
 PeriodicSite: Cu (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
 PeriodicSite: Cu (1.95, 1.95, 6.05) [0.5, 0.5, 0.5]
 PeriodicSite: O (1.95, 0.0, 1.194e-16) [0.5, 0.0, 0.0]
 PeriodicSite: O (-1.194e-16, 1.95, 1.194e-16) [0.0, 0.5, 0.0]
 PeriodicSite: O (-1.194e-16, 1.95, 6.05) [0.0, 0.5, 0.5]
 PeriodicSite: O (1.95, 0.0, 6.05) [0.5, 0.0, 0.5]
 PeriodicSite: O (-1.194e-16, 1.95, 3.025) [0.0, 0.5, 0.25]
 PeriodicSite: O (1.95, 0.0, 3.025) [0.5, 0.0, 0.25]
 

In [127]:
wyckoffgene

{'spacegroup': 227,
 'species': ['In', 'Sn', 'Ag', 'S'],
 'sites': [['16d'], ['16d'], ['8a'], ['32e']],
 'numIons': [16, 16, 8, 32],
 'lattice':  10.7000,  10.7000,  10.7000,  90.0000,  90.0000,  90.0000, cubic,
 'occupancy': defaultdict(<function __main__.generate_wyckoffgene.<locals>.<lambda>()>,
             {'In': defaultdict(list, {'16d': [0.5]}),
              'Sn': defaultdict(list, {'16d': [0.5]}),
              'Ag': defaultdict(list, {'8a': [1.0]}),
              'S': defaultdict(list, {'32e': [1.0]})}),
 'frac_coord': defaultdict(<function __main__.generate_wyckoffgene.<locals>.<lambda>()>,
             {'In': defaultdict(list, {'16d': [array([0.75, 0.75, 0.5 ])]}),
              'Sn': defaultdict(list, {'16d': [array([0.75, 0.75, 0.5 ])]}),
              'Ag': defaultdict(list, {'8a': [array([0.125, 0.625, 0.625])]}),
              'S': defaultdict(list, {'32e': [array([0.  , 0.25, 0.75])]})}),
 'structure_pymatgen': Structure Summary
 Lattice
     abc : 10.7 10.7 10.7
  an

In [108]:
pymatgen_structure = Structure.from_str(cif_lines,fmt='cif')
sga = SpacegroupAnalyzer(pymatgen_structure,symprec=0.0001)
#refined_struc = sga.get_refined_structure()
#sga = SpacegroupAnalyzer(refined_struc,symprec=0.01)
structure = sga.get_symmetrized_structure()
structure, structure.distance_matrix.shape, np.min(structure.distance_matrix + np.diag(np.ones(structure.distance_matrix.shape[0]) * (cutoff + 10.)) )

/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['international']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['number']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['equivalent_atoms']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['wyckoffs']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}

(SymmetrizedStructure
 Full Formula (Sm1 Co5.4 Ni3.6)
 Reduced Formula: Sm1Co5.4Ni3.6
 Spacegroup: P6/mmm (191)
 abc   :   4.900000   4.900000   4.000000
 angles:  90.000000  90.000000 120.000000
 Sites (10)
   #  SP                  a      b    c  Wyckoff
 ---  --------------  -----  -----  ---  ---------
   0  Sm              0      0      0    1a
   1  Co:0.6, Ni:0.4  0.667  0.333  0    6l
   2  Co:0.6, Ni:0.4  0.5    0.5    0.5  3g,
 (10, 10),
 0.004899999999999991)

In [126]:
cif_lines.split('\n')

['data_',
 '',
 "_symmetry_space_group_name_H-M 'Fd-3m'",
 '_symmetry_Int_Tables_number                  227',
 '_symmetry_cell_setting                     cubic',
 '_cell_length_a              10.700',
 '_cell_length_b              10.700',
 '_cell_length_c              10.700',
 '_cell_angle_alpha           90.000',
 '_cell_angle_beta            90.000',
 '_cell_angle_gamma           90.000',
 '_cell_volume              1225.043',
 '',
 'loop_',
 ' _symmetry_equiv_pos_site_id',
 ' _symmetry_equiv_pos_as_xyz',
 "1 'x, y, z'",
 "2 '-x+3/4, -y+1/4, z+1/2'",
 "3 '-x+1/4, y+1/2, -z+3/4'",
 "4 'x+1/2, -y+3/4, -z+1/4'",
 "5 'z, x, y'",
 "6 'z+1/2, -x+3/4, -y+1/4'",
 "7 '-z+3/4, -x+1/4, y+1/2'",
 "8 '-z+1/4, x+1/2, -y+3/4'",
 "9 'y, z, x'",
 "10 '-y+1/4, z+1/2, -x+3/4'",
 "11 'y+1/2, -z+3/4, -x+1/4'",
 "12 '-y+3/4, -z+1/4, x+1/2'",
 "13 'y+3/4, x+1/4, -z+1/2'",
 "14 '-y, -x, -z'",
 "15 'y+1/4, -x+1/2, z+3/4'",
 "16 '-y+1/2, x+3/4, z+1/4'",
 "17 'x+3/4, z+1/4, -y+1/2'",
 "18 '-x+1/2, z+3/4, y

In [125]:
count_test = 0
for wyckoffgene in all_wyckoffgenes:
    cif_lines = get_cif_lines(wyckoffgene)
    pymatgen_structure = Structure.from_str(cif_lines,fmt='cif')
    cif_file = StringIO(cif_lines)
    atoms = read(cif_file, format="cif")
    structure = AseAtomsAdaptor.get_structure(atoms)
    if not structure_validity(structure):
        if count_test == 1:
            break
        else:
            count_test += 1
structure.distance_matrix.shape, np.min(structure.distance_matrix + np.diag(np.ones(structure.distance_matrix.shape[0]) * (cutoff + 10.)) )

/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/pymatgen/io/cif.py:1229: UserWarning: Issues encountered while parsing CIF: Skipping relative stoichiometry check because CIF does not contain formula keys.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/ase/io/cif.py:408: UserWarning: crystal system 'tetragonal' is not interpreted for space group Spacegroup(139, setting=1). This may result in wrong setting!
  warnings.warn(
/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/pymatgen/io/cif.py:1229: UserWarning: Issues encountered while parsing CIF: 1 fractional coordinates rounded to ideal values to avoid issues with finite precision.
Skipping relative stoichiometry check because CIF does not contain formula keys.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/home/energy/mahpe/anaconda3/envs/env_

ValueError: Invalid CIF file with no structures!

In [129]:
def split_wyckoff(site):
    """
    Split the Wyckoff site into multiplicity and letter.
    
    Args:
        site (str): The Wyckoff site string (e.g., '2a', '3b').
    
    Returns:
        tuple: A tuple containing the multiplicity (int) and the letter (str).
    """
    for i, c in enumerate(site):
        if c.isalpha():
            return int(site[:i]), site[i:]
        


def get_cif_lines(wyckoffgene_dict):
    """
    Generate CIF lines from a wyckoffgene dictionary.

    Args:
        wyckoffgene_dict (dict): Dictionary containing crystal structure information with keys 'spacegroup_group', 'lattice', 'sites', 'species', 'frac_coord', and 'occupancy'.
    
    Returns:
        str: A string containing the CIF lines.
    """

    # Initiate the cif lines
    lines = ""
    # Symmetry information
    spg_group = wyckoffgene_dict['spacegroup_group']
    l_type =spg_group.lattice_type
    number = spg_group.number
    G1 = spg_group[0]
    symbol =spg_group.symbol #if G1.is_standard_setting()

    lines += "data_\n"

    lines += f"\n_symmetry_space_group_name_H-M '{symbol:s}'\n"
    lines += f"_symmetry_Int_Tables_number      {number:>15d}\n"
    lines += f"_symmetry_cell_setting           {l_type:>15s}\n"

    a, b, c, alpha, beta, gamma = wyckoffgene_dict['lattice'].get_para(degree=True)
    lines += f"_cell_length_a        {a}\n"
    lines += f"_cell_length_b        {b}\n"
    lines += f"_cell_length_c        {c}\n"
    lines += f"_cell_angle_alpha     {alpha}\n"
    lines += f"_cell_angle_beta      {beta}\n"
    lines += f"_cell_angle_gamma     {gamma}\n"
    lines += f"_cell_volume          {wyckoffgene_dict['lattice'].volume}\n"
    lines += "\nloop_\n"
    lines += " _symmetry_equiv_pos_site_id\n"
    lines += " _symmetry_equiv_pos_as_xyz\n"
    for i, op in enumerate(G1):
        lines += f"{i + 1:d} '{op.as_xyz_str():s}'\n"

    lines += "\nloop_\n"
    lines += " _atom_site_label\n"
    lines += " _atom_site_type_symbol\n"
    lines += " _atom_site_symmetry_multiplicity\n"
    lines += " _atom_site_Wyckoff_symbol\n" # ICSD style
    lines += " _atom_site_fract_x\n"
    lines += " _atom_site_fract_y\n"
    lines += " _atom_site_fract_z\n"
    lines += " _atom_site_occupancy\n"
    #print(wyckoffgene_dict['sites'])
    count= {}
    for i in range(len(wyckoffgene_dict['sites'])):
        wp_list = wyckoffgene_dict['sites'][i]
        for wp_site in wp_list:
            # Split the Wyckoff site into multiplicity and letter
            mul, letter = split_wyckoff(wp_site)
            
            specie = wyckoffgene_dict["species"][i]
            coord = wyckoffgene_dict['frac_coord'][specie][wp_site]
            occ = wyckoffgene_dict['occupancy'][specie][wp_site]

            if len(coord) >1: # if there are 2 frac coordinates at the same site for the same element. Exp. S is at 6c, twich
                if specie not in count.keys():
                    count[specie] = {wp_site: 0}
                else:
                    if wp_site not in count[specie].keys():
                        count[specie][wp_site] = 0

                coord_i = coord[count[specie][wp_site]]
                occ_i = occ[count[specie][wp_site]]
                count[specie][wp_site] += 1
            else:
                coord_i = coord[0]
                occ_i = occ[0]


            lines += f"{specie:6s} {specie:6s} {mul:3d} {letter:s}"
            lines += f" {coord_i[0]} {coord_i[1]} {coord_i[2]}"
            lines += f" {occ_i} \n"

            
    lines += "#END\n\n"
    return lines
cif_lines = get_cif_lines(wyckoffgene)
cif_lines.split('\n')

['data_',
 '',
 "_symmetry_space_group_name_H-M 'Fd-3m'",
 '_symmetry_Int_Tables_number                  227',
 '_symmetry_cell_setting                     cubic',
 '_cell_length_a        10.7',
 '_cell_length_b        10.7',
 '_cell_length_c        10.7',
 '_cell_angle_alpha     90.0',
 '_cell_angle_beta      90.0',
 '_cell_angle_gamma     90.0',
 '_cell_volume          1225.043',
 '',
 'loop_',
 ' _symmetry_equiv_pos_site_id',
 ' _symmetry_equiv_pos_as_xyz',
 "1 'x, y, z'",
 "2 '-x+3/4, -y+1/4, z+1/2'",
 "3 '-x+1/4, y+1/2, -z+3/4'",
 "4 'x+1/2, -y+3/4, -z+1/4'",
 "5 'z, x, y'",
 "6 'z+1/2, -x+3/4, -y+1/4'",
 "7 '-z+3/4, -x+1/4, y+1/2'",
 "8 '-z+1/4, x+1/2, -y+3/4'",
 "9 'y, z, x'",
 "10 '-y+1/4, z+1/2, -x+3/4'",
 "11 'y+1/2, -z+3/4, -x+1/4'",
 "12 '-y+3/4, -z+1/4, x+1/2'",
 "13 'y+3/4, x+1/4, -z+1/2'",
 "14 '-y, -x, -z'",
 "15 'y+1/4, -x+1/2, z+3/4'",
 "16 '-y+1/2, x+3/4, z+1/4'",
 "17 'x+3/4, z+1/4, -y+1/2'",
 "18 '-x+1/2, z+3/4, y+1/4'",
 "19 '-x, -z, -y'",
 "20 'x+1/4, -z+1/2, y+3

In [71]:
# save the cif files 
open('cif_lines.cif', 'w').write(cif_lines)

1114

In [131]:
cutoff = 0.5
dist_mat = structure.distance_matrix
# Pad diagonal with a large number
dist_mat = np.min(structure.distance_matrix + np.diag(np.ones(structure.distance_matrix.shape[0]) * (cutoff + 10.)) )
dist_mat.min()

1.9266168539294453

In [ ]:
from dis_gen.generation import generate_cif_files
len_all_structures = len(save_dict['spacegroup'])
all_cifs,cifs__wyckoffgene = generate_cif_files(all_wyckoffgenes,maxiter=len_all_structures,validity_primitive=False,symmetry_analyzer=False, verbose=False, two_oxidation_state=False)

Processing 27008 structures out of 27008


  0%|          | 0/27008 [00:00<?, ?it/s]/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/ase/io/cif.py:408: UserWarning: crystal system 'tetragonal' is not interpreted for space group Spacegroup(139, setting=1). This may result in wrong setting!
  warnings.warn(
/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/ase/io/cif.py:408: UserWarning: crystal system 'cubic' is not interpreted for space group Spacegroup(227, setting=1). This may result in wrong setting!
  warnings.warn(
/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/ase/io/cif.py:408: UserWarning: crystal system 'hexagonal' is not interpreted for space group Spacegroup(173, setting=1). This may result in wrong setting!
  warnings.warn(
/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/ase/io/cif.py:408: UserWarning: crystal system 'orthorhombic' is not interpreted for space group Spacegroup(36, setting=1). This may result in wrong settin

Failed to parse 0 structures out of 27008 Procentage: 100.00%
Validity check failed for 6957 structures out of 27008 Procentage: 74.24%
Oxidation state check failed for 9603 structures out of 27008 Procentage: 64.44%
Total valid structures: 15061 out of 27008 Procentage: 55.76%


In [20]:
with open('/home/energy/mahpe/Published_code/Dis-gen/New_icsd_structure.json','r') as f:
    data_dict= json.load(f)

In [21]:
count_valid = 0

for i in tqdm(data_dict):
    # from pymatgen dict 
    structure = Structure.from_dict(data_dict[i])
    if not structure_validity(structure):
        count_valid += 1
print(f"Number of valid structures: {len(data_dict) - count_valid} out of {len(data_dict)}, Procentage: {100-(count_valid)/len(data_dict)*100:.2f}%")

Number of valid structures: 207948 out of 217593, Procentage: 95.57%


In [ ]:
# load structures
with open('/home/energy/mahpe/Published_code/Dis-gen/New_sym_structures_icsd_all_new.pkl', 'rb') as f:
    data = pickle.load(f)


In [18]:
count_valid = 0
for test_struc in tqdm(data):
    if not structure_validity(test_struc):
        count_valid += 1
print(f"Number of valid structures: {len(data) - count_valid} out of {len(data)}, Procentage: {100-(count_valid)/len(data)*100:.2f}%")

100%|██████████| 135038/135038 [00:25<00:00, 5294.01it/s]


Number of valid structures: 132250 out of 135038, Procentage: 97.94%


In [6]:
failed_count = 0
for i in range(len(decode_dict['spacegroup'])):
        spacegroup_int = decode_dict['spacegroup'][i]
        spacegroup_group = Group(spacegroup_int)
        index_max = np.argmax(decode_dict['wyckoff_letter'][i], axis=1)
        index = index_max != 0
        
        # Define the Wyckoff sites letters and multipliers            
        w_letter = np.array([chr(ord('a') + l - 1) for l in np.argmax(decode_dict['wyckoff_letter'][i][index], axis=1) if l != 0])
        w_multiplier = np.array([m for m in np.argmax(decode_dict['wyckoff_mult'][i][index], axis=1) if m != 0])
        w_site = [str(w) + l for w, l in zip(w_multiplier, w_letter)]

        # Possible wyckoff sites Symmetry accucary 
        for vae_letter, vae_mult in zip(w_letter, w_multiplier):
            try:
                mult_true = spacegroup_group.get_wp_by_letter(vae_letter).multiplicity
            except IndexError:
                symmetry_accuracy = False
                break
            if mult_true != vae_mult:      
                symmetry_accuracy = False
                break 
            else: 
                symmetry_accuracy = True
        if not symmetry_accuracy:
            print(f"Symmetry accuracy failed for index {i} with spacegroup {spacegroup_int} and Wyckoff sites {w_site}")
            failed_count += 1
            continue

### Generation

In [ ]:
def align_fractional_coordinates(fractional_coord,wp_site,spg,spg_group):
    """
    Align the fractional coordinates to the Wyckoff position.
    
    Args:
        fractional_coord (np.ndarray): The fractional coordinates to align.
        wp_site (str): The Wyckoff position site (e.g., '2a', '3b').
        spg (int): The space group number.
        spg_group (pyxtal.symmetry.Group): The space group object.
    
    Returns:
        np.ndarray: The aligned fractional coordinates.
        pyxtal.symmetry.WyckoffPosition: The Wyckoff position object.
        int: The orbit index of the Wyckoff position.
    """
    wp = SG_TO_WP_TO_SITE_SYMM[spg][wp_site]

    closes =[]
    for orbit_index in range(len(wp.ops)):
        close = search_cloest_wp(spg_group, wp, wp.ops[orbit_index], fractional_coord)
        closes.append((close, wp, orbit_index, np.linalg.norm(np.minimum((close - fractional_coord)%1., (fractional_coord - close)%1.)) ))

    # Get the closest fractional coordinates
    closest = sorted(closes, key=lambda x: x[-1])[0]
    return closest[0], closest[1], closest[2]  # Return the closest fractional coordinates, Wyckoff position, and orbit index 


def generate_wyckoffgene(data_pkl, max_iter = None,element_acc = 0.01,disorder_acc = 0.1,shift_frac_coord=True,verbose = False):
    """
    Process the data from a dictionary containing crystal structure information and return a list of symmetrized structures.
    Args:
        data_pkl (dict): Dictionary containing crystal structure data with keys 'abc', 'angles', 'spacegroup', 'element', 'wyckoff_letter', 'wyckoff_mult', 'frac_coords', and 'disordered_site'.
        max_iter (int, optional): Maximum number of structures to process. If None, all structures are processed.
        element_acc (float, optional): Minimum fraction for an element to be considered present.
        disorder_acc (float, optional): Threshold for determining if a site is disordered.
        shift_frac_coord (bool, optional): If True, shift fractional coordinates to align with Wyckoff positions.
        verbose (bool, optional): If True, print additional information during processing.
    Returns:
        list: A list of wyckoffgene dictionaries, each containing information about a crystal structure.
        int: Number of symmetry matching accuracy failures.
        int: Number of failed items due to invalid lattice parameters or other issues.
    """

    wyckoffgenes_list = [] 
    failed_count = 0  # Counter for failed items
    SMA_count = 0  # Counter for symmetry matching accuracy failures
    for i in tqdm(range(len(data_pkl['abc']))):
        # # Create the structure            
        spacegroup_int = data_pkl['spacegroup'][i]
        spacegroup_group = SG_SYM[spacegroup_int] if spacegroup_int in SG_SYM else Group(spacegroup_int)     
        
        # Define the lattice parameters
        lattice = Lattice.from_parameters(
            a=data_pkl['abc'][i][0],
            b=data_pkl['abc'][i][1],
            c=data_pkl['abc'][i][2],
            alpha=data_pkl['angles'][i][0],
            beta=data_pkl['angles'][i][1],
            gamma=data_pkl['angles'][i][2]
        )
        # Check if the lattice is valid
        if any(np.array(lattice.abc) == 0):
            if verbose:
                print(f"Invalid lattice parameters for index {i}: {lattice.abc}")
            failed_count += 1
            continue
        # create lattic object
        # Identify if a Wyckoff site is empty
        index_max = np.argmax(data_pkl['wyckoff_letter'][i], axis=1)
        index = index_max != 0

        # Define if the site is disordered
        disordered_site = data_pkl['disordered_site'][i][index]
        disordered_site = disordered_site>disorder_acc

        # Loop trough all wyckoff sites and define the element
        index_array = np.arange(0,len(index))[index]
        element_comb = []
        for site, a_list in enumerate(data_pkl['element'][i][index]):

            if disordered_site[site]: # Disordered
                element_index = np.where(a_list > element_acc)[0] + 1
                disordered = True
            else: # Ordered
                element_index = np.argmax(a_list) + 1
                disordered = False                    
            
            if disordered:
                disordered_element = {chemical_symbols[elem]: round(float(data_pkl['element'][i][site][elem - 1]),2) for elem in element_index if chemical_symbols[elem] !='Md'} # Md is vacancies
                if len(disordered_element) == 0:  # If no elements are present, skip this site
                    index[index_array[site]] = False  # Mark the site as not present
                else:
                    element_comb.append(disordered_element)
            else:
                if chemical_symbols[element_index] == 'Md':  # If the element is Md, it is a vacancy
                    index[index_array[site]] = False  # Mark the site as not present
                else:
                    element_comb.append({chemical_symbols[element_index]: 1.0})
        
        # Define the Fractional coordinates
        frac_coords = data_pkl['frac_coords'][i][index]
        print(element_comb)
        # Define the Wyckoff sites letters and multipliers            
        w_letter = np.array([chr(ord('a') + l - 1) for l in np.argmax(data_pkl['wyckoff_letter'][i][index], axis=1) if l != 0])
        w_multiplier = np.array([m for m in np.argmax(data_pkl['wyckoff_mult'][i][index], axis=1) if m != 0])
        w_site = [str(w) + l for w, l in zip(w_multiplier, w_letter)]

        # Check if the length of Wyckoff letters and multipliers match. A mismatch can happen if either the letter or multiplier is zero.
        if len(w_letter) != len(w_multiplier):
            if verbose:
                print(index)
                print(w_letter, w_multiplier,w_site)
                print(f"Mismatch in length of Wyckoff letters and multipliers for index {i}.")
            failed_count += 1
            continue
        
        # Check if the symmetry matching is correct
        symm_acc = Symmetry_matching_accuracy(w_letter, w_multiplier, spacegroup_group)
        if not symm_acc:
            if verbose:
                print(f"Symmetry accuracy failed for index {i} with spacegroup {spacegroup_int} and Wyckoff sites {w_site}")
            SMA_count += 1
            continue
        if verbose:
            print(f"Symmetry accuracy passed for index {i} with spacegroup {spacegroup_int} and Wyckoff sites {w_site}")

        # Shift fractional coordinates if needed 
        if shift_frac_coord:
            coord_list = []
            for wp_site, coord in zip(w_site, frac_coords):

                # Extra check for symmetry accuracy
                if wp_site not in SG_TO_WP_TO_SITE_SYMM[spacegroup_int].keys():
                    if verbose:
                        print(f"Wyckoff site {wp_site} not found in spacegroup {spacegroup_int}.")
                    SMA_count += 1
                    symm_acc = False
                    break 

                # Align the fractional coordinates to the Wyckoff position
                coord, _, _ = align_fractional_coordinates(coord, wp_site, spacegroup_int, spacegroup_group)
                coord_list.append(coord)
            if not symm_acc:
                continue 

            frac_coords = coord_list
        
        # Get the wyckoffgene 
        element_wyckoff = defaultdict(list)
        element_counts = defaultdict(int)
        elements_occ = defaultdict(lambda: defaultdict(list))
        elements_frac_coord = defaultdict(lambda: defaultdict(list))

        for el_dict, wyck, fr in zip(element_comb, w_site, frac_coords):
            for el, occ in el_dict.items():
                element_wyckoff[el].append(wyck)
                element_counts[el] += int(round(int(''.join(filter(str.isdigit, wyck)))))  # assume multiplicity from Wyckoff symbol
                elements_occ[el][wyck].append(occ)  # accumulate occupancy for each element and wyckoff site
                elements_frac_coord[el][wyck].append(fr)  # accumulate fractional coordinates for each elemen        

        # Step 2: Build wyckoffgene
        wyckoffgene = {
            'spacegroup': spacegroup_int,
            'species': list(element_wyckoff.keys()),
            'sites': list(element_wyckoff.values()),
            'numIons': list(element_counts.values()),
            'lattice':px.lattice.Lattice(ltype=spacegroup_group.lattice_type,
                        volume=lattice.volume,
                        matrix=lattice.matrix
                        ),
            'occupancy': elements_occ,  # Assuming full occupancy for simplicity    
            'frac_coord': elements_frac_coord,
            'structure_pymatgen': Structure(lattice, element_comb, frac_coords),
            'spacegroup_group':spacegroup_group,
        }

        wyckoffgenes_list.append(wyckoffgene)
        if max_iter is not None and i >= max_iter:
            break
    return wyckoffgenes_list, SMA_count, failed_count
# Decode the samples to structures
SG_SYM = {spacegroup: Group(spacegroup) for spacegroup in range(1, 231)}
SG_TO_WP_TO_SITE_SYMM = dict()
for spacegroup in range(1, 231):
    SG_TO_WP_TO_SITE_SYMM[spacegroup] = dict()
    for wp in SG_SYM[spacegroup].Wyckoff_positions:
        wp_site = str(wp.multiplicity)+wp.letter
        wp.get_site_symmetry()
        SG_TO_WP_TO_SITE_SYMM[spacegroup][wp_site] = wp
#       SG_TO_WP_TO_SITE_SYMM[spacegroup][wp_site][wp] = wp.get_site_symmetry_object().to_one_hot()

# Generate Wyckoff genes
mixiter = None
element_acc = 0.05  
disorder_acc = 0.5
all_wyckoffgenes, sma_int, failed_int = generate_wyckoffgene(save_dict,
                             max_iter=mixiter,
                             element_acc=element_acc,
                             disorder_acc=disorder_acc,
                             shift_frac_coord=True,
                             verbose=False)
if mixiter is not None:
    print(f"Symmetry Matching Accuracy: {100-sma_int/mixiter*100:.2f}%")
    print(f"Number of failed items: {failed_int} of {mixiter} processed, Procentage: {failed_int/mixiter*100:.2f}%")
else:
    print(f"Symmetry Matching Accuracy: {100- sma_int/len(save_dict['spacegroup'])*100:.2f}%")
    print(f"Number of failed items: {failed_int} of {len(save_dict['spacegroup'])} processed, Procentage: {failed_int/len(save_dict['spacegroup'])*100:.2f}%")


100%|██████████| 27008/27008 [59:59<00:00,  7.50it/s]  

Symmetry Matching Accuracy: 98.39%
Number of failed items: 3 of 27008 processed, Procentage: 0.01%



KeyboardInterrupt



In [9]:
(sma_int/27008)*100

1.6069312796208532

In [117]:
def split_wyckoff(site):
    """
    Split the Wyckoff site into multiplicity and letter.
    
    Args:
        site (str): The Wyckoff site string (e.g., '2a', '3b').
    
    Returns:
        tuple: A tuple containing the multiplicity (int) and the letter (str).
    """
    for i, c in enumerate(site):
        if c.isalpha():
            return int(site[:i]), site[i:]
        


def get_cif_lines(wyckoffgene_dict):
    """
    Generate CIF lines from a wyckoffgene dictionary.

    Args:
        wyckoffgene_dict (dict): Dictionary containing crystal structure information with keys 'spacegroup_group', 'lattice', 'sites', 'species', 'frac_coord', and 'occupancy'.
    
    Returns:
        str: A string containing the CIF lines.
    """

    # Initiate the cif lines
    lines = ""
    # Symmetry information
    spg_group = wyckoffgene_dict['spacegroup_group']
    l_type =spg_group.lattice_type
    number = spg_group.number
    G1 = spg_group[0]
    symbol =spg_group.symbol #if G1.is_standard_setting()

    lines += "data_\n"

    lines += f"\n_symmetry_space_group_name_H-M '{symbol:s}'\n"
    lines += f"_symmetry_Int_Tables_number      {number:>15d}\n"
    lines += f"_symmetry_cell_setting           {l_type:>15s}\n"

    a, b, c, alpha, beta, gamma = wyckoffgene_dict['lattice'].get_para(degree=True)
    lines += f"_cell_length_a        {a:12.3f}\n"
    lines += f"_cell_length_b        {b:12.3f}\n"
    lines += f"_cell_length_c        {c:12.3f}\n"
    lines += f"_cell_angle_alpha     {alpha:12.3f}\n"
    lines += f"_cell_angle_beta      {beta:12.3f}\n"
    lines += f"_cell_angle_gamma     {gamma:12.3f}\n"
    lines += f"_cell_volume          {wyckoffgene_dict['lattice'].volume:12.3f}\n"
    lines += "\nloop_\n"
    lines += " _symmetry_equiv_pos_site_id\n"
    lines += " _symmetry_equiv_pos_as_xyz\n"
    for i, op in enumerate(G1):
        lines += f"{i + 1:d} '{op.as_xyz_str():s}'\n"

    lines += "\nloop_\n"
    lines += " _atom_site_label\n"
    lines += " _atom_site_type_symbol\n"
    lines += " _atom_site_symmetry_multiplicity\n"
    lines += " _atom_site_Wyckoff_symbol\n" # ICSD style
    lines += " _atom_site_fract_x\n"
    lines += " _atom_site_fract_y\n"
    lines += " _atom_site_fract_z\n"
    lines += " _atom_site_occupancy\n"
    #print(wyckoffgene_dict['sites'])
    count= {}
    for i in range(len(wyckoffgene_dict['sites'])):
        wp_list = wyckoffgene_dict['sites'][i]
        for wp_site in wp_list:
            # Split the Wyckoff site into multiplicity and letter
            mul, letter = split_wyckoff(wp_site)
            
            specie = wyckoffgene_dict["species"][i]
            coord = wyckoffgene_dict['frac_coord'][specie][wp_site]
            occ = wyckoffgene_dict['occupancy'][specie][wp_site]

            if len(coord) >1: # if there are 2 frac coordinates at the same site for the same element. Exp. S is at 6c, twich
                if specie not in count.keys():
                    count[specie] = {wp_site: 0}
                else:
                    if wp_site not in count[specie].keys():
                        count[specie][wp_site] = 0

                coord_i = coord[count[specie][wp_site]]
                occ_i = occ[count[specie][wp_site]]
                count[specie][wp_site] += 1
            else:
                coord_i = coord[0]
                occ_i = occ[0]


            lines += f"{specie:6s} {specie:6s} {mul:3d} {letter:s}"
            lines += f"{coord_i[0]} {coord_i[1]} {coord_i[2]}"
            lines += f" {occ_i} \n"

            
    lines += "#END\n\n"
    return lines


In [17]:
from io import StringIO
len_all_structures = len(save_dict['spacegroup'])

def generate_cif_files(all_wyckoffgenes_list,maxiter=None, validity_check=True,charge_neutral=True,
                       two_oxidation_state=False,validity_primitive=False,symmetry_analyzer=False, verbose=False):
    """
    Generate CIF files from a list of WyckoffGene dictionaries.
    Parameters:
    all_wyckoffgenes_list (list): List of WyckoffGene dictionaries.
    maxiter (int): Maximum number of structures to process. If None, process all.
    validity_check (bool): If True, check the validity of the structures.
    charge_neutral (bool): If True, check the oxidation state validity.
    two_oxidation_state (bool): If True, allow two oxidation states.
    validity_primitive (bool): If True, use the primitive structure for validity checks.
    symmetry_analyzer (bool): If True, use pymatgen's symmetry analyzer.
    verbose (bool): If True, print additional information.
    Returns:
    all_cif_lines (list): List of CIF lines for valid structures.
    cif_lines_wyckoffgene (list): List of CIF lines with corresponding WyckoffGene dictionaries.
    """

    all_cif_lines = []
    cif_lines_wyckoffgene = []
    failed_count = 0
    sym_count = 0
    validity_count = 0
    oxidation_state_count = 0
    validity_count_list = []

    struc_valid = True # If they are not chossen as input, they are valid by default
    oxidation_valid = True # If they are not chossen as input, they are valid by default
    if maxiter is not None:
        print(f"Processing {maxiter} structures out of {len(all_wyckoffgenes_list)}")

    for i in tqdm(range(len(all_wyckoffgenes_list))):
        if maxiter is not None:
            if i >= maxiter:
                break
        wyckoffgene = all_wyckoffgenes_list[i]

        cif_lines = get_cif_lines(wyckoffgene)
        

        try:
            # primitve cell
            if validity_primitive:
                structure = wyckoffgene['structure_pymatgen']
            # Unit cell
            else:
                # Found that pymatgen had some unknown issues readin some cif files, while ase had no errors. Since the efficiency only increased by 10% we will do it with ase
                #structure = Structure.from_str(cif_lines,fmt='cif')
                cif_file = StringIO(cif_lines)
                atoms = read(cif_file, format="cif")
                structure = AseAtomsAdaptor.get_structure(atoms)


        except:
            failed_count += 1
            continue

        if symmetry_analyzer:
            try:
                sga = SpacegroupAnalyzer(structure,symprec=0.1)
                refined_struc = sga.get_refined_structure()
                sga = SpacegroupAnalyzer(refined_struc,symprec=0.01)
                structure = sga.get_symmetrized_structure()
            except:
                sym_count +=1
                continue
        
        if validity_check:
            struc_valid =structure_validity(structure)
            if not struc_valid:
                validity_count += 1
        
        if charge_neutral:   
            oxidation_valid = oxidation_state_validity(structure,two_oxidation_state=two_oxidation_state,verbose=verbose)
            if not oxidation_valid:
                oxidation_state_count += 1
    
        # if one of them is not valid, skip the structures
        if not struc_valid:
            continue 
        if not oxidation_valid:
            continue

        # Add the structure to the list
        all_cif_lines.append(cif_lines)
        cif_lines_wyckoffgene.append(wyckoffgene)

    if maxiter is not None:
        print(f'Failed to parse {failed_count} structures out of {maxiter} Procentage: {100-failed_count/maxiter*100:.2f}%')
        if symmetry_analyzer:
            print(f'Pymatgen Symmetry failed for {sym_count} structures out of {maxiter} Procentage: {100-sym_count/maxiter*100:.2f}%')
        if validity_check:
            print(f'Validity check failed for {validity_count} structures out of {maxiter} Procentage: {100-validity_count/maxiter*100:.2f}%')
        if charge_neutral:
            print(f'Oxidation state check failed for {oxidation_state_count} structures out of {maxiter} Procentage: {100-oxidation_state_count/maxiter*100:.2f}%')
        print(f'Total valid structures: {len(all_cif_lines)} out of {maxiter} Procentage: {len(all_cif_lines)/maxiter*100:.2f}%')
    else:
        print(f"Failed to parse {failed_count} structures out of {len(all_wyckoffgenes_list)} Procentage: {100-failed_count/len(all_wyckoffgenes_list)*100:.2f}%")
        if symmetry_analyzer:
            print(f"Pymatgen Symmetry failed for {sym_count} structures out of {len(all_wyckoffgenes_list)} Procentage: {100-sym_count/len(all_wyckoffgenes_list)*100:.2f}%")
        if validity_check:
            print(f"Validity check failed for {validity_count} structures out of {len(all_wyckoffgenes_list)} Procentage: {100-validity_count/len(all_wyckoffgenes_list)*100:.2f}%")
        if charge_neutral:
            print(f"Oxidation state check failed for {oxidation_state_count} structures out of {len(all_wyckoffgenes_list)} Procentage: {oxidation_state_count/len(all_wyckoffgenes_list)*100:.2f}%")
        print(f"Total valid structures: {len(all_cif_lines)} out of {len(all_wyckoffgenes_list)} Procentage: {len(all_cif_lines)/len(all_wyckoffgenes_list)*100:.2f}%")    

    return all_cif_lines, cif_lines_wyckoffgene


In [20]:
len_all_structures = len(save_dict['spacegroup'])


all_cifs,cifs__wyckoffgene = generate_cif_files(all_wyckoffgenes,maxiter=len_all_structures,validity_primitive=False,symmetry_analyzer=False, verbose=False)

Processing 27008 structures out of 26571


100%|██████████| 26571/26571 [07:49<00:00, 56.65it/s] 

Failed to parse 0 structures out of 27008 Procentage: 100.00%
Validity check failed for 10462 structures out of 27008 Procentage: 61.26%
Oxidation state check failed for 10104 structures out of 27008 Procentage: 62.59%
Total valid structures: 12392 out of 27008 Procentage: 45.88%


In [ ]:

all_cif_lines = []
cif_lines_wyckoffgene = []
failed_count = 0
sym_count = 0
validity_count = 0
oxidation_state_count = 0
validity_count_list = []

maxiter = None

for i in tqdm(range(len(all_wyckoffgenes))):
    if maxiter is not None:
        if i >= maxiter:
            break
    wyckoffgene = all_wyckoffgenes[i]
    # check if the structure is valid
    occ = wyckoffgene['occupancy']
    disordered = any_not_one = any(
    value != 1.0
    for element in occ.values()
    for site_values in element.values()
    for value in site_values
    ) 

    cif_lines = get_cif_lines(wyckoffgene)
    
    try:
        # Use StringIO to simulate a file
        #cif_file = StringIO(cif_lines)

        #structure = wyckoffgene['structure_pymatgen']
        # Read using ASE
        #strucre = CifParser(cif_file).get_structures()[0]
        structure = Structure.from_str(cif_lines,fmt='cif')
        #atoms = read(cif_file, format="cif")
        #structure = AseAtomsAdaptor.get_structure(atoms)
    except:
        failed_count += 1
        continue
    #try:
        #structure = Structure.from_str(cif_lines,fmt='cif')
    #    sga = SpacegroupAnalyzer(structure,symprec=0.1)
    #    refined_struc = sga.get_refined_structure()
    #    sga = SpacegroupAnalyzer(refined_struc,symprec=0.01)
    #    structure = sga.get_symmetrized_structure()
    #except:
    #    sym_count +=1
    #    continue
    
    struc_valid =structure_validity(structure,cutoff=0.2)
    oxidation_valid = oxidation_state_validity(structure,two_oxidation_state=False,verbose=False)

    if not struc_valid:
        validity_count += 1
    if not oxidation_valid:
        oxidation_state_count += 1
    # if one of them is not valid, skip the structure
    if not struc_valid:
        continue 
    if not oxidation_valid:
        continue
    # Add the structure to the list
    all_cif_lines.append(cif_lines)
    cif_lines_wyckoffgene.append(wyckoffgene)
    
if maxiter is not None:
    print(f'Failed to parse {failed_count} structures out of {maxiter} Procentage: {failed_count/maxiter*100:.2f}%')
    print(f'Pymatgen Symmetry failed for {sym_count} structures out of {maxiter} Procentage: {sym_count/maxiter*100:.2f}%')
    print(f'Validity check failed for {validity_count} structures out of {maxiter} Procentage: {validity_count/maxiter*100:.2f}%')
    print(f'Oxidation state check failed for {oxidation_state_count} structures out of {maxiter} Procentage: {oxidation_state_count/maxiter*100:.2f}%')
    print(f'Total valid structures: {len(all_cif_lines)} out of {maxiter} Procentage: {len(all_cif_lines)/maxiter*100:.2f}%')
else:
    print(f"Failed to parse {failed_count} structures out of {len(all_wyckoffgenes)} Procentage: {failed_count/len(all_wyckoffgenes)*100:.2f}%")
    print(f"Pymatgen Symmetry failed for {sym_count} structures out of {len(all_wyckoffgenes)} Procentage: {sym_count/len(all_wyckoffgenes)*100:.2f}%")
    print(f"Validity check failed for {validity_count} structures out of {len(all_wyckoffgenes)} Procentage: {validity_count/len(all_wyckoffgenes)*100:.2f}%")
    print(f"Oxidation state check failed for {oxidation_state_count} structures out of {len(all_wyckoffgenes)} Procentage: {oxidation_state_count/len(all_wyckoffgenes)*100:.2f}%")
    print(f"Total valid structures: {len(all_cif_lines)} out of {len(all_wyckoffgenes)} Procentage: {len(all_cif_lines)/len(all_wyckoffgenes)*100:.2f}%")


100%|██████████| 9848/9848 [00:26<00:00, 370.15it/s]

Failed to parse 0 structures out of 9848 Procentage: 0.00%
Pymatgen Symmetry failed for 0 structures out of 9848 Procentage: 0.00%
Validity check failed for 242 structures out of 9848 Procentage: 2.46%
Oxidation state check failed for 4540 structures out of 9848 Procentage: 46.10%
Total valid structures: 5175 out of 9848 Procentage: 52.55%


In [97]:
all_wyckoffgenes[161]


{'spacegroup': 227,
 'species': ['Li', 'Ti', 'O'],
 'sites': [['8a', '16d'], ['16d'], ['32e']],
 'numIons': [24, 16, 32],
 'lattice':   8.3700,   8.3800,   8.3800,  90.0000,  90.0000,  90.1000, cubic,
 'occupancy': defaultdict(<function __main__.generate_wyckoffgene.<locals>.<lambda>()>,
             {'Li': defaultdict(list, {'8a': [1.0], '16d': [0.1]}),
              'Ti': defaultdict(list, {'16d': [0.82]}),
              'O': defaultdict(list, {'32e': [1.0]})}),
 'frac_coord': defaultdict(<function __main__.generate_wyckoffgene.<locals>.<lambda>()>,
             {'Li': defaultdict(list,
                          {'8a': [array([0.125, 0.625, 0.625])],
                           '16d': [array([0.75, 0.75, 0.5 ])]}),
              'Ti': defaultdict(list, {'16d': [array([0.75, 0.75, 0.5 ])]}),
              'O': defaultdict(list, {'32e': [array([0.25, 0.  , 0.75])]})}),
 'structure_pymatgen': Structure Summary
 Lattice
     abc : 8.36999988555909 8.380000114440927 8.380000114440918
  ang

In [91]:
CifParser(cif_file).get_structures()[0]

/tmp/ipykernel_2825457/1912692584.py:1: FutureWarning: get_structures is deprecated
get_structures is deprecated and will be removed in 2024. Use parse_structures instead.The only difference is that primitive defaults to False in the new parse_structures method.So parse_structures(primitive=True) is equivalent to the old behavior of get_structures().
  CifParser(cif_file).get_structures()[0]


ValueError: Invalid CIF file with no structures!

In [90]:
cif_lines.split('\n')

['data_',
 '',
 "_symmetry_space_group_name_H-M 'Fd-3m'",
 '_symmetry_Int_Tables_number                  227',
 '_symmetry_cell_setting                     cubic',
 '_cell_length_a               8.200',
 '_cell_length_b               8.220',
 '_cell_length_c               7.940',
 '_cell_angle_alpha           90.000',
 '_cell_angle_beta            90.000',
 '_cell_angle_gamma           90.100',
 '_cell_volume               535.187',
 '',
 'loop_',
 ' _symmetry_equiv_pos_site_id',
 ' _symmetry_equiv_pos_as_xyz',
 "1 'x, y, z'",
 "2 '-x+3/4, -y+1/4, z+1/2'",
 "3 '-x+1/4, y+1/2, -z+3/4'",
 "4 'x+1/2, -y+3/4, -z+1/4'",
 "5 'z, x, y'",
 "6 'z+1/2, -x+3/4, -y+1/4'",
 "7 '-z+3/4, -x+1/4, y+1/2'",
 "8 '-z+1/4, x+1/2, -y+3/4'",
 "9 'y, z, x'",
 "10 '-y+1/4, z+1/2, -x+3/4'",
 "11 'y+1/2, -z+3/4, -x+1/4'",
 "12 '-y+3/4, -z+1/4, x+1/2'",
 "13 'y+3/4, x+1/4, -z+1/2'",
 "14 '-y, -x, -z'",
 "15 'y+1/4, -x+1/2, z+3/4'",
 "16 '-y+1/2, x+3/4, z+1/4'",
 "17 'x+3/4, z+1/4, -y+1/2'",
 "18 '-x+1/2, z+3/4, y

In [65]:
Group(227)

-- Spacegroup --# 227 (Fd-3m)--
192i	site symm: 1
96h	site symm: ..2
96g	site symm: ..m
48f	site symm: 2.mm
32e	site symm: .3m
16d	site symm: .-3m
16c	site symm: .-3m
8b	site symm: -43m
8a	site symm: -43m

In [66]:

candidate = px.pyxtal()
candidate.from_random(
    dim=3,
    group=wyckoffgene["spacegroup"],
    species=wyckoffgene["species"],
    numIons=wyckoffgene["numIons"],
    sites=wyckoffgene["sites"],
    tm=Tol_matrix(prototype="atomic", factor=1.3),
    max_count= 100,
    lattice=wyckoffgene["lattice"],
)

# change the fractional coordiantes:
frac_change = True
if frac_change:
    for i in range(len(candidate.atom_sites)):
        specie = candidate.atom_sites[i].specie
        wp = str(candidate.atom_sites[i].wp.multiplicity)+ str(candidate.atom_sites[i].wp.letter)
        print(specie, wp, wyckoffgene['frac_coord'][specie][wp])
        candidate.atom_sites[i].position = list(wyckoffgene['frac_coord'][specie][wp])

#spg = SpacegroupAnalyzer(candidate.to_pymatgen(), symprec=0.1)
#conventional_structure = spg.get_refined_structure()
#spg = SpacegroupAnalyzer(conventional_structure,symprec=0.01)
#struc = spg.get_symmetrized_structure()
#candidate.to_file('test.cif', fmt='cif')
candidate.to_pymatgen()

KeyboardInterrupt: 

In [ ]:

all_cif_lines = []
cif_lines_wyckoffgene = []
failed_count = 0
validity_count = 0
oxidation_state_count = 0
validity_count_list = []

maxiter = None

for i in tqdm(range(len(all_wyckoffgenes))):
    if maxiter is not None:
        if i >= maxiter:
            break
    wyckoffgene = all_wyckoffgenes[i]
    # check if the structure is valid
    cif_lines = get_cif_lines(wyckoffgene)
    
    structure = Structure.from_str(cif_lines,fmt='cif')
    try:
        #structure = Structure.from_str(cif_lines,fmt='cif')
        sga = SpacegroupAnalyzer(structure,symprec=0.1)
        refined_struc = sga.get_refined_structure()
        sga = SpacegroupAnalyzer(refined_struc,symprec=0.01)
        structure = sga.get_symmetrized_structure()
    except:
        failed_count +=1
        continue
    
    struc_valid =structure_validity(structure)
    oxidation_valid = oxidation_state_validity(structure,two_oxidation_state=False,verbose=False)

    if not struc_valid:
        validity_count += 1
    if not oxidation_valid:
        oxidation_state_count += 1
    # if one of them is not valid, skip the structure
    if not (struc_valid and oxidation_valid):
        continue
    # Add the structure to the list
    all_cif_lines.append(cif_lines)
    cif_lines_wyckoffgene.append(wyckoffgene)
    
if maxiter is not None:
    print(f'Failed to parse {failed_count} structures out of {maxiter} Procentage: {failed_count/maxiter*100:.2f}%')
    print(f'Validity check failed for {validity_count} structures out of {maxiter} Procentage: {validity_count/maxiter*100:.2f}%')
    print(f'Oxidation state check failed for {oxidation_state_count} structures out of {maxiter} Procentage: {oxidation_state_count/maxiter*100:.2f}%')
    print(f'Total valid structures: {len(all_cif_lines)} out of {maxiter} Procentage: {len(all_cif_lines)/maxiter*100:.2f}%')
else:
    print(f"Failed to parse {failed_count} structures out of {len(all_wyckoffgenes)} Procentage: {failed_count/len(all_wyckoffgenes)*100:.2f}%")
    print(f"Validity check failed for {validity_count} structures out of {len(all_wyckoffgenes)} Procentage: {validity_count/len(all_wyckoffgenes)*100:.2f}%")
    print(f"Oxidation state check failed for {oxidation_state_count} structures out of {len(all_wyckoffgenes)} Procentage: {oxidation_state_count/len(all_wyckoffgenes)*100:.2f}%")
    print(f"Total valid structures: {len(all_cif_lines)} out of {len(all_wyckoffgenes)} Procentage: {len(all_cif_lines)/len(all_wyckoffgenes)*100:.2f}%")


In [ ]:
all_cif_lines = []
failed_count = 0
validity_count = 0
oxidation_state_count = 0

maxiter = None
for i in tqdm(np.arange(0,len(indexes))[indexes]):
    
    if maxiter is not None:
        if i >= maxiter:
            break
    cif_str =df_test['cif'].values[i]
    #cif_file = StringIO(cif_str)
    # Parse the CIF
    #parser = CifParser(cif_file)
    #structure = parser.get_structures()[0] 
    try:
        structure = Structure.from_str(cif_str,fmt='cif')
        sga = SpacegroupAnalyzer(structure,symprec=0.1)
        refined_struc = sga.get_refined_structure()
        sga = SpacegroupAnalyzer(refined_struc,symprec=0.01)
        structure = sga.get_symmetrized_structure()
        structure.remove_oxidation_states()
    except:
        failed_count +=1
        continue
    
    if not structure_validity(structure):
        validity_count += 1
        continue
    if not oxidation_state_validity(structure,two_oxidation_state=False,verbose=False):
        oxidation_state_count += 1
        continue
    # Add the structure to the list
    all_cif_lines.append(cif_str)

if maxiter is not None:
    print(f'Failed to parse {failed_count} structures out of {maxiter} Procentage: {failed_count/maxiter*100:.2f}%')
    print(f'Validity check failed for {validity_count} structures out of {maxiter} Procentage: {validity_count/maxiter*100:.2f}%')
    print(f'Oxidation state check failed for {oxidation_state_count} structures out of {maxiter} Procentage: {oxidation_state_count/maxiter*100:.2f}%')
    print(f'Total valid structures: {len(all_cif_lines)} out of {maxiter} Procentage: {len(all_cif_lines)/maxiter*100:.2f}%')
else:
    print(f"Failed to parse {failed_count} structures out of {len(df_test['cif'].values)} Procentage: {failed_count/len(all_wyckoffgenes)*100:.2f}%")
    print(f"Validity check failed for {validity_count} structures out of {len(df_test['cif'].values)} Procentage: {validity_count/len(all_wyckoffgenes)*100:.2f}%")
    print(f"Oxidation state check failed for {oxidation_state_count} structures out of {len(all_wyckoffgenes)} Procentage: {oxidation_state_count/len(all_wyckoffgenes)*100:.2f}%")
    print(f"Total valid structures: {len(all_cif_lines)} out of {len(all_wyckoffgenes)} Procentage: {len(all_cif_lines)/len(all_wyckoffgenes)*100:.2f}%")

  0%|          | 18/26571 [00:00<11:25, 38.76it/s]/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/pymatgen/io/cif.py:1141: UserWarning: Incorrect stoichiometry:
  CIF={'Ba': 0.6, 'Cu': 1.0, 'La': 1.0, 'O': 5.15, 'Sr': 0.4, 'Tl': 1.0}
  PMG={'La': 0.998, 'Ba': 0.6, 'Sr': 0.4, 'Tl': 0.99, 'Cu': 1.0, 'O': 5.154}
  ratios={'La': 0.998, 'O': 1.000776699029126, 'Tl': 0.99, 'Sr': 1.0, 'Cu': 1.0, 'Ba': 1.0}
  warnings.warn(cif_failure_reason)
/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/pymatgen/io/cif.py:1141: UserWarning: Incorrect stoichiometry:
  CIF={'Br': 0.73, 'Cl': 3.27, 'K': 2.0, 'Pd': 1.0}
  PMG={'K': 2.0, 'Pd': 1.0, 'Br': 0.72, 'Cl': 3.28}
  ratios={'Cl': 1.003058103975535, 'K': 1.0, 'Pd': 1.0, 'Br': 0.9863013698630136}
  warnings.warn(cif_failure_reason)
  0%|          | 24/26571 [00:00<10:05, 43.83it/s]/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/pymatgen/io/cif.py:1141: UserWarning: Missing elements C f

Failed to parse 0 structures out of 27008 Procentage: 0.00%
Validity check failed for 541 structures out of 27008 Procentage: 2.04%
Oxidation state check failed for 7359 structures out of 26571 Procentage: 27.70%
Total valid structures: 18671 out of 26571 Procentage: 70.27%


In [39]:
all_cif_lines[999].split('\n')

['data_',
 '',
 "_symmetry_space_group_name_H-M 'Cmcm'",
 '_symmetry_Int_Tables_number                   63',
 '_symmetry_cell_setting              orthorhombic',
 '_cell_length_a            4.200000',
 '_cell_length_b           11.300000',
 '_cell_length_c            4.500000',
 '_cell_angle_alpha        90.000003',
 '_cell_angle_beta         90.000003',
 '_cell_angle_gamma        90.800002',
 '_cell_volume            213.549176',
 '',
 'loop_',
 ' _symmetry_equiv_pos_site_id',
 ' _symmetry_equiv_pos_as_xyz',
 "1 'x, y, z'",
 "2 '-x, -y, z+1/2'",
 "3 '-x, y, -z+1/2'",
 "4 'x, -y, -z'",
 "5 '-x, -y, -z'",
 "6 'x, y, -z+1/2'",
 "7 'x, -y, z+1/2'",
 "8 '-x, y, z'",
 "9 'x+1/2, y+1/2, z'",
 "10 '-x+1/2, -y+1/2, z+1/2'",
 "11 '-x+1/2, y+1/2, -z+1/2'",
 "12 'x+1/2, -y+1/2, -z'",
 "13 '-x+1/2, -y+1/2, -z'",
 "14 'x+1/2, y+1/2, -z+1/2'",
 "15 'x+1/2, -y+1/2, z+1/2'",
 "16 '-x+1/2, y+1/2, z'",
 '',
 'loop_',
 ' _atom_site_label',
 ' _atom_site_type_symbol',
 ' _atom_site_symmetry_multiplicity'

In [26]:
cif_dict = {}
for i, cif_lines in enumerate(all_cif_lines):
    cif_dict[i] = cif_lines
# Save the cif_dict to a file
with open('cif_dict.json', 'w') as f:
    json.dump(cif_dict, f)

In [27]:
# read
with open('cif_dict.json', 'r') as f:
    cif_dict = json.load(f)

In [36]:
cif_dict['1000'].split('\n')

['data_',
 '',
 "_symmetry_space_group_name_H-M 'Cmcm'",
 '_symmetry_Int_Tables_number                   63',
 '_symmetry_cell_setting              orthorhombic',
 '_cell_length_a            4.200000',
 '_cell_length_b           11.300000',
 '_cell_length_c            4.500000',
 '_cell_angle_alpha        90.000003',
 '_cell_angle_beta         90.000003',
 '_cell_angle_gamma        90.800002',
 '_cell_volume            213.549176',
 '',
 'loop_',
 ' _symmetry_equiv_pos_site_id',
 ' _symmetry_equiv_pos_as_xyz',
 "1 'x, y, z'",
 "2 '-x, -y, z+1/2'",
 "3 '-x, y, -z+1/2'",
 "4 'x, -y, -z'",
 "5 '-x, -y, -z'",
 "6 'x, y, -z+1/2'",
 "7 'x, -y, z+1/2'",
 "8 '-x, y, z'",
 "9 'x+1/2, y+1/2, z'",
 "10 '-x+1/2, -y+1/2, z+1/2'",
 "11 '-x+1/2, y+1/2, -z+1/2'",
 "12 'x+1/2, -y+1/2, -z'",
 "13 '-x+1/2, -y+1/2, -z'",
 "14 'x+1/2, y+1/2, -z+1/2'",
 "15 'x+1/2, -y+1/2, z+1/2'",
 "16 '-x+1/2, y+1/2, z'",
 '',
 'loop_',
 ' _atom_site_label',
 ' _atom_site_type_symbol',
 ' _atom_site_symmetry_multiplicity'

In [ ]:
cif_str =df_test['cif'].values[i+1]
pymatgen_struc = Structure.from_str(cif_str,fmt='cif')
sga = SpacegroupAnalyzer(pymatgen_struc,symprec=0.1)
refined_struc = sga.get_refined_structure()
sga = SpacegroupAnalyzer(refined_struc,symprec=0.01)
strc_symmetry = sga.get_symmetrized_structure()
spacegroup_pymatgen = sga.get_space_group_number()
strc_symmetry

/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['international']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['number']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['equivalent_atoms']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['wyckoffs']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}

Composition('Ca2+:18 Mn4+:6 Zn2+:6 O2-:36')

In [162]:
open('test.cif', 'w').write(cif_lines)
open('test_true.cif', 'w').write(cif_str)

6567

In [143]:
wyckoffgene = all_wyckoffgenes[i-1]
cif_lines = get_cif_lines(wyckoffgene)
cif_file = StringIO(cif_lines)
# Parse the CIF
parser = CifParser(cif_file)
structure = parser.get_structures()[0] 
sga = SpacegroupAnalyzer(structure,symprec=0.1)
refined_struc = sga.get_refined_structure()
sga = SpacegroupAnalyzer(refined_struc,symprec=0.01)
strc_symmetry = sga.get_symmetrized_structure()
spacegroup_pymatgen = sga.get_space_group_number()


/tmp/ipykernel_588734/1767655176.py:6: FutureWarning: get_structures is deprecated
get_structures is deprecated and will be removed in 2024. Use parse_structures instead.The only difference is that primitive defaults to False in the new parse_structures method.So parse_structures(primitive=True) is equivalent to the old behavior of get_structures().
  structure = parser.get_structures()[0]


TypeError: cannot unpack non-iterable NoneType object

In [45]:
cif_lines.split('\n')

['data_',
 '',
 "_symmetry_space_group_name_H-M 'I4/mmm'",
 '_symmetry_Int_Tables_number                  139',
 '_symmetry_cell_setting                tetragonal',
 '_cell_length_a            2.954988',
 '_cell_length_b            2.954988',
 '_cell_length_c           20.902544',
 '_cell_angle_alpha        90.000000',
 '_cell_angle_beta         90.000000',
 '_cell_angle_gamma        90.000000',
 '_cell_volume            182.520009',
 '',
 'loop_',
 ' _symmetry_equiv_pos_site_id',
 ' _symmetry_equiv_pos_as_xyz',
 "1 'x, y, z'",
 "2 '-x, -y, z'",
 "3 '-y, x, z'",
 "4 'y, -x, z'",
 "5 '-x, y, -z'",
 "6 'x, -y, -z'",
 "7 'y, x, -z'",
 "8 '-y, -x, -z'",
 "9 '-x, -y, -z'",
 "10 'x, y, -z'",
 "11 'y, -x, -z'",
 "12 '-y, x, -z'",
 "13 'x, -y, z'",
 "14 '-x, y, z'",
 "15 '-y, -x, z'",
 "16 'y, x, z'",
 "17 'x+1/2, y+1/2, z+1/2'",
 "18 '-x+1/2, -y+1/2, z+1/2'",
 "19 '-y+1/2, x+1/2, z+1/2'",
 "20 'y+1/2, -x+1/2, z+1/2'",
 "21 '-x+1/2, y+1/2, -z+1/2'",
 "22 'x+1/2, -y+1/2, -z+1/2'",
 "23 'y+1/2, 

In [ ]:

wyckoffgene = all_wyckoffgenes[0]  # Use the first wyckoffgene for testing


candidate = px.pyxtal()
candidate.from_random(
    dim=3,
    group=wyckoffgene["spacegroup"],
    species=wyckoffgene["species"],
    numIons=wyckoffgene["numIons"],
    sites=wyckoffgene["sites"],
    tm=Tol_matrix(prototype="atomic", factor=1.3),
    max_count= 100,
    lattice=wyckoffgene["lattice"],
)

# change the fractional coordiantes:
frac_change = False
if frac_change:
    for i in range(len(candidate.atom_sites)):
        specie = candidate.atom_sites[i].specie
        wp = str(candidate.atom_sites[i].wp.multiplicity)+ str(candidate.atom_sites[i].wp.letter)
        print(specie, wp)
        candidate.atom_sites[i].position = wyckoffgene['frac_coord'][specie][wp]

#spg = SpacegroupAnalyzer(candidate.to_pymatgen(), symprec=0.1)
#conventional_structure = spg.get_refined_structure()
#spg = SpacegroupAnalyzer(conventional_structure,symprec=0.01)
#struc = spg.get_symmetrized_structure()
#candidate.to_file('test.cif', fmt='cif')
candidate


------Crystal from random------
Dimension: 3
Composition: Ce4Nd4Cu2O8
Group: I 4/m m m (139)
  2.9550,   2.9550,  20.9025,  90.0000,  90.0000,  90.0000, tetragonal
Wyckoff sites:
	Ce @ [ 0.0000  0.0000  0.6159], WP [4e] Site m]
	Nd @ [ 0.0000  0.0000  0.7972], WP [4e] Site m]
	Cu @ [ 0.0000  0.0000  0.0000], WP [2a] Site [4/mmm]
	 O @ [ 0.0000  0.5000  0.0000], WP [4c] Site [mmm.]
	 O @ [ 0.0000  0.5000  0.2500], WP [4d] Site [-4m2]

In [ ]:

def split_wyckoff(site):
    for i, c in enumerate(site):
        if c.isalpha():
            return int(site[:i]), site[i:]
def get_cif_lines(wyckoffgene_dict, shift_frac_coord=True):

    # Initiate the cif lines
    lines = ""
    # Symmetry information
    G = wyckoffgene_dict['spacegroup_group']
    l_type =G.lattice_type
    number = G.number
    G1 = G[0]
    symbol =G.symbol if G1.is_standard_setting()

    lines += "data_\n"

    lines += f"\n_symmetry_space_group_name_H-M '{symbol:s}'\n"
    lines += f"_symmetry_Int_Tables_number      {number:>15d}\n"
    lines += f"_symmetry_cell_setting           {l_type:>15s}\n"

    a, b, c, alpha, beta, gamma = wyckoffgene_dict['lattice'].get_para(degree=True)
    lines += f"_cell_length_a        {a:12.6f}\n"
    lines += f"_cell_length_b        {b:12.6f}\n"
    lines += f"_cell_length_c        {c:12.6f}\n"
    lines += f"_cell_angle_alpha     {alpha:12.6f}\n"
    lines += f"_cell_angle_beta      {beta:12.6f}\n"
    lines += f"_cell_angle_gamma     {gamma:12.6f}\n"
    lines += f"_cell_volume          {wyckoffgene_dict['lattice'].volume:12.6f}\n"
    lines += "\nloop_\n"
    lines += " _symmetry_equiv_pos_site_id\n"
    lines += " _symmetry_equiv_pos_as_xyz\n"

    for i, op in enumerate(G1):
        lines += f"{i + 1:d} '{op.as_xyz_str():s}'\n"

    lines += "\nloop_\n"
    lines += " _atom_site_label\n"
    lines += " _atom_site_type_symbol\n"
    lines += " _atom_site_symmetry_multiplicity\n"
    lines += " _atom_site_Wyckoff_symbol\n" # ICSD style
    lines += " _atom_site_fract_x\n"
    lines += " _atom_site_fract_y\n"
    lines += " _atom_site_fract_z\n"
    lines += " _atom_site_occupancy\n"

    for i in range(len(wyckoffgene['sites'])):
        wp_list = wyckoffgene['sites'][i]

        for wp_site in wp_list:
            # Split the Wyckoff site into multiplicity and letter
            mul, letter = split_wyckoff(wp_site)
            
            specie = wyckoffgene_dict["species"][i]
            coord = wyckoffgene_dict['frac_coord'][specie][wp_site][0]
            occ = wyckoffgene_dict['occupancy'][specie][wp_site][0]

            #if shift_frac_coord:
                # Align the fractional coordinates to the Wyckoff position
            #    coord, _, _ = align_fractional_coordinates(coord, wp_site, wyckoffgene_dict['spacegroup'], wyckoffgene_dict['spacegroup_group'])

            lines += f"{specie:6s} {specie:6s} {mul:3d} {letter:s}"
            lines += "{:12.6f}{:12.6f}{:12.6f}".format(*coord)
            lines += f" {occ} \n"
    lines += "#END\n\n"
    return lines

#for i in tqdm(range(len(all_wyckoffgenes))):
#    wyckoffgene = all_wyckoffgenes[i]
#    cif_lines = get_cif_lines(wyckoffgene, shift_frac_coord=False)

#cif_lines_list = [get_cif_lines(wyckoffgene,shift_frac_coord=True) for wyckoffgene in all_wyckoffgenes]

idx = 1
wyckoffgene = all_wyckoffgenes[idx]
cif_lines = get_cif_lines(wyckoffgene,shift_frac_coord=True)

cif_file = StringIO(cif_lines)

# Parse the CIF
parser = CifParser(cif_file)
structure = parser.get_structures()[0] 
print(structure_validity(structure),structure.distance_matrix.shape)

True (22, 22)


/tmp/ipykernel_575149/474235019.py:81: FutureWarning: get_structures is deprecated
get_structures is deprecated and will be removed in 2024. Use parse_structures instead.The only difference is that primitive defaults to False in the new parse_structures method.So parse_structures(primitive=True) is equivalent to the old behavior of get_structures().
  structure = parser.get_structures()[0]
/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/pymatgen/io/cif.py:1229: UserWarning: Issues encountered while parsing CIF: Skipping relative stoichiometry check because CIF does not contain formula keys.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [33]:
wyckoffgene['frac_coord'] = {'In':{'16d':np.array([0.875, 0.875, 0.625])},'Sn':{'16d':np.array([0.875, 0.875, 0.625])},
                             'Ag':{'8a':np.array([0.25, 0.75, 0.75])}, 'S':{'32a':np.array([0.136, 0.136, 0.864])}}

In [ ]:
structure_validity(strc_symmetry)

True

In [37]:
all_wyckoffgenes[idx]

{'spacegroup': 227,
 'species': ['In', 'Sn', 'Ag', 'S'],
 'sites': [['16d'], ['16d'], ['8a'], ['32e']],
 'numIons': [16, 16, 8, 32],
 'lattice':  10.8000,  10.8000,  10.9000,  90.0000,  90.0000,  89.8000, cubic,
 'occupancy': defaultdict(<function __main__.process_data.<locals>.<lambda>()>,
             {'In': defaultdict(list, {'16d': [0.45]}),
              'Sn': defaultdict(list, {'16d': [0.51]}),
              'Ag': defaultdict(list, {'8a': [1.0]}),
              'S': defaultdict(list, {'32e': [1.0]})}),
 'frac_coord': {'In': {'16d': array([0.875, 0.875, 0.625])},
  'Sn': {'16d': array([0.875, 0.875, 0.625])},
  'Ag': {'8a': array([0.25, 0.75, 0.75])},
  'S': {'32a': array([0.136, 0.136, 0.864])}},
 'frac_coord_wyckoff': {'16d': array([0.88837653, 0.7131648 , 0.6879745 ], dtype=float32),
  '8a': array([0.30108595, 0.8416567 , 0.5831241 ], dtype=float32),
  '32e': array([0.13387603, 0.13729098, 0.9486795 ], dtype=float32)},
 'spacegroup_group': -- Spacegroup --# 227 (Fd-3m)--
 192i	

In [23]:
cif_str =df_test['cif'].values[idx]
pymatgen_struc = Structure.from_str(cif_str,fmt='cif')
sga = SpacegroupAnalyzer(pymatgen_struc,symprec=0.1)
refined_struc = sga.get_refined_structure()
sga = SpacegroupAnalyzer(refined_struc,symprec=0.01)
strc_symmetry = sga.get_symmetrized_structure()
spacegroup_pymatgen = sga.get_space_group_number()
strc_symmetry

/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['international']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['number']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['equivalent_atoms']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['wyckoffs']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}

SymmetrizedStructure
Full Formula (In8 Ag8 Sn8 S32)
Reduced Formula: InAgSnS4
Spacegroup: Fd-3m (227)
abc   :  10.740000  10.740000  10.740000
angles:  90.000000  90.000000  90.000000
Sites (56)
  #  SP                      a      b      c  Wyckoff
---  ------------------  -----  -----  -----  ---------
  0  In3+:0.5, Sn4+:0.5  0.875  0.875  0.625  16d
  1  Ag+                 0.25   0.75   0.75   8a
  2  S2-                 0.136  0.136  0.864  32e

### Tests

In [ ]:
from pyxtal.symmetry import search_cloest_wp, Group
idx = 2
wyckoffgene = all_wyckoffgenes[0]


spg = wyckoffgene['spacegroup']
spg_group = wyckoffgene['spacegroup_group']
species = wyckoffgene['species'][idx]

for wp_site in  wyckoffgene['sites'][idx]:
    print('wp_site:',wp_site)
    frac_coord = np.array(wyckoffgene['frac_coord'][species][wp_site][0])
    closes_frac = []
    wp = SG_TO_WP_TO_SITE_SYMM[spg][wp_site]

    closes =[]
    for orbit_index in range(len(wp.ops)):
        print(orbit_index)
        close = search_cloest_wp(spg_group, wp, wp.ops[orbit_index], frac_coord)
        closes.append((close, wp, orbit_index, np.linalg.norm(np.minimum((close - frac_coord)%1., (frac_coord - close)%1.))))
closest = sorted(closes, key=lambda x: x[-1])[0]


In [116]:
from pyxtal.symmetry import get_wyckoffs

Group(62).get_wp_by_letter('l').multiplicity

IndexError: list index out of range

In [10]:
Group(62).__dir__()

['string',
 'dim',
 'header',
 'symbol',
 'number',
 'PBC',
 'lattice_type',
 'lattice_id',
 'point_group',
 'pg_number',
 'polar',
 'inversion',
 'chiral',
 'hall_number',
 'P',
 'P1',
 'wyckoffs',
 'w_symm',
 'Wyckoff_positions',
 'wyckoffs_organized',
 '__module__',
 '__doc__',
 '__init__',
 '_initialize_hall_data',
 '_initialize_wyckoff_data',
 '__str__',
 '__repr__',
 '__iter__',
 '__getitem__',
 '__len__',
 'get_ferroelectric_groups',
 'get_site_dof',
 'get_spg_representation',
 'get_lattice_id',
 'get_lattice_dof',
 'is_valid_combination',
 'list_wyckoff_combinations',
 'get_spg_symmetry_object',
 'get_wyckoff_position',
 'get_wyckoff_position_from_xyz',
 'get_alternatives',
 '_get_max_k_subgroup',
 '_get_max_t_subgroup',
 'get_max_k_subgroup',
 'get_max_t_subgroup',
 'get_max_subgroup',
 'get_wp_list',
 'get_splitters_from_structure',
 'get_splitters_from_relation',
 'get_min_supergroup',
 '_get_max_subgroup_numbers',
 'get_max_subgroup_numbers',
 'get_lists',
 'get_lists_mol',

In [68]:
all_dict['element'][0]

[{'Ce': 0.06, 'Nd': 0.88}, {'Cu': 1.0}, {'O': 1.0}, {'O': 1.0}]

In [18]:
import pyxtal as px

index = 0
elements = all_dict['element'][index]
frac_coord = all_dict['frac_coord'][index]
wyckoff_sites = all_dict['wyckoff_site'][index]
sp = all_dict['spacegroup'][index]
print('Element:', elements, 'Wyckoff sites:', wyckoff_sites, 'Spacegroup:', sp)

from collections import defaultdict
# Step 1: Group wyckoff sites by element
element_wyckoff = defaultdict(list)
element_counts = defaultdict(int)
elements_occ = defaultdict(lambda: defaultdict(list))
elements_frac_coord = defaultdict(lambda: defaultdict(list))

for el_dict, wyck, fr in zip(elements, wyckoff_sites, frac_coord):
    for el, occ in el_dict.items():
        element_wyckoff[el].append(wyck)
        element_counts[el] += int(round(int(''.join(filter(str.isdigit, wyck)))))  # assume multiplicity from Wyckoff symbol
        elements_occ[el][wyck].append(occ)  # accumulate occupancy for each element and wyckoff site
        elements_frac_coord[el][wyck].append(fr)  # accumulate fractional coordinates for each elemen        

# Step 2: Build wyckoffgene
wyckoffgene = {
    'group': sp,
    'species': list(element_wyckoff.keys()),
    'sites': list(element_wyckoff.values()),
    'numIons': list(element_counts.values()),
    'lattice':px.lattice.Lattice(ltype=Group(sp).lattice_type,
                   volume=all_dict['lattice'][index].volume,
                   matrix=all_dict['lattice'][index].matrix
                   ),
    'occupancy': elements_occ,  # Assuming full occupancy for simplicity    
    'frac_coord': elements_frac_coord,
}

wyckoffgene

Element: [{'Ce': 0.06, 'Nd': 0.88}, {'Cu': 1.0}, {'O': 1.0}, {'O': 1.0}] Wyckoff sites: ['4e', '2a', '4c', '4d'] Spacegroup: 139


{'group': 139,
 'species': ['Ce', 'Nd', 'Cu', 'O'],
 'sites': [['4e'], ['4e'], ['2a'], ['4c', '4d']],
 'numIons': [4, 4, 2, 8],
 'lattice':   3.9000,   3.9000,  12.0000,  90.0000,  90.0000,  90.0000, tetragonal,
 'occupancy': defaultdict(<function __main__.<lambda>()>,
             {'Ce': defaultdict(list, {'4e': [0.06]}),
              'Nd': defaultdict(list, {'4e': [0.88]}),
              'Cu': defaultdict(list, {'2a': [1.0]}),
              'O': defaultdict(list, {'4c': [1.0], '4d': [1.0]})}),
 'frac_coord': defaultdict(<function __main__.<lambda>()>,
             {'Ce': defaultdict(list,
                          {'4e': [array([0.5341019 , 0.41357625, 0.30256617], dtype=float32)]}),
              'Nd': defaultdict(list,
                          {'4e': [array([0.5341019 , 0.41357625, 0.30256617], dtype=float32)]}),
              'Cu': defaultdict(list,
                          {'2a': [array([ 0.00978125,  0.01976012, -0.01626195], dtype=float32)]}),
              'O': defaultdict(

In [53]:
import pyxtal as px
from pyxtal.symmetry import Group
from pyxtal.tolerance import Tol_matrix
iadm = Tol_matrix(prototype="atomic", factor=1.3)
nlimit = 1000

dim = 3
spg: int = wyckoffgene["group"]
species_in: list[str] = wyckoffgene["species"]
numIons_in: list[int] = wyckoffgene["numIons"]
Wyckoff_in: list[list] = wyckoffgene["sites"]
lattice: px.lattice.Lattice = wyckoffgene["lattice"]

candidate = px.pyxtal()
candidate.from_random(
    dim=dim,
    group=spg,
    species=species_in,
    numIons=numIons_in,
    sites=Wyckoff_in,
    tm=iadm,
    max_count=nlimit,
    lattice=lattice,
)

# change the fractional coordiantes:
frac_change = False
if frac_change:
    for i in range(len(candidate.atom_sites)):
        specie = candidate.atom_sites[i].specie
        wp = str(candidate.atom_sites[i].wp.multiplicity)+ str(candidate.atom_sites[i].wp.letter)
        candidate.atom_sites[i].position = wyckoffgene['frac_coord'][specie][wp]

#spg = SpacegroupAnalyzer(candidate.to_pymatgen(), symprec=0.1)
#conventional_structure = spg.get_refined_structure()
#spg = SpacegroupAnalyzer(conventional_structure,symprec=0.01)
#struc = spg.get_symmetrized_structure()
#candidate.to_file('test.cif', fmt='cif',style='icsd')

KeyError: 'group'

In [ ]:
def get_cif_lines(structure, wyckoffgene_dict):
    """
    Generate CIF lines from a candidate structure and wyckoffgene information.
    Args:
        structure (pyxtal.pyxtal): The candidate structure.
        wyckoff_dict (dict): Dictionary containing wyckoff information.
    Returns:
        str: CIF lines as a string.
    """

    # Initiate the cif lines
    lines = ""

    # Symmetry information
    l_type = structure.group.lattice_type
    number = structure.group.number
    G1 = structure.group[0]
    symbol = structure.group.symbol if G1.is_standard_setting() else sites[0].wp.get_hm_symbol()

    lines += "data_\n"

    lines += f"\n_symmetry_space_group_name_H-M '{symbol:s}'\n"
    lines += f"_symmetry_Int_Tables_number      {number:>15d}\n"
    lines += f"_symmetry_cell_setting           {l_type:>15s}\n"

    a, b, c, alpha, beta, gamma = structure.lattice.get_para(degree=True)
    lines += f"_cell_length_a        {a:12.6f}\n"
    lines += f"_cell_length_b        {b:12.6f}\n"
    lines += f"_cell_length_c        {c:12.6f}\n"
    lines += f"_cell_angle_alpha     {alpha:12.6f}\n"
    lines += f"_cell_angle_beta      {beta:12.6f}\n"
    lines += f"_cell_angle_gamma     {gamma:12.6f}\n"
    lines += f"_cell_volume          {structure.lattice.volume:12.6f}\n"
    lines += "\nloop_\n"
    lines += " _symmetry_equiv_pos_site_id\n"
    lines += " _symmetry_equiv_pos_as_xyz\n"

    for i, op in enumerate(G1):
        lines += f"{i + 1:d} '{op.as_xyz_str():s}'\n"

    lines += "\nloop_\n"
    lines += " _atom_site_label\n"
    lines += " _atom_site_type_symbol\n"
    lines += " _atom_site_symmetry_multiplicity\n"
    lines += " _atom_site_Wyckoff_symbol\n" # ICSD style
    lines += " _atom_site_fract_x\n"
    lines += " _atom_site_fract_y\n"
    lines += " _atom_site_fract_z\n"
    lines += " _atom_site_occupancy\n"

    for site in structure.atom_sites:
        print(site)
        mul = site.wp.multiplicity
        letter = site.wp.letter
        wp = str(site.wp.multiplicity)+ str(site.wp.letter)
        specie = site.specie
        coord = wyckoffgene_dict['frac_coord'][site.specie][wp][0]
        occ = wyckoffgene_dict['occupancy'][site.specie][wp][0]
        lines += f"{specie:6s} {specie:6s} {mul:3d} {letter:s}"
        lines += "{:12.6f}{:12.6f}{:12.6f}".format(*coord)
        lines += f" {occ} \n"
    lines.split('\n')
    lines += "#END\n\n"
    return lines
get_cif_lines(candidate, wyckoffgene).split('\n')



Ce @ [ 0.0000  0.0000  0.1836], WP [4e] Site m]


IndexError: list index out of range

In [55]:
line_test

['data_',
 '',
 "_symmetry_space_group_name_H-M 'P63'",
 '_symmetry_Int_Tables_number                  173',
 '_symmetry_cell_setting                 hexagonal',
 '_cell_length_a           10.000000',
 '_cell_length_b           10.100000',
 '_cell_length_c            6.000000',
 '_cell_angle_alpha        90.000003',
 '_cell_angle_beta         90.000003',
 '_cell_angle_gamma       120.199991',
 '_cell_volume            523.750598',
 '',
 'loop_',
 ' _symmetry_equiv_pos_site_id',
 ' _symmetry_equiv_pos_as_xyz',
 "1 'x, y, z'",
 "2 '-y, x-y, z'",
 "3 '-x+y, -x, z'",
 "4 '-x, -y, z+1/2'",
 "5 'y, -x+y, z+1/2'",
 "6 'x-y, x, z+1/2'",
 '',
 'loop_',
 ' _atom_site_label',
 ' _atom_site_type_symbol',
 ' _atom_site_symmetry_multiplicity',
 ' _atom_site_Wyckoff_symbol',
 ' _atom_site_fract_x',
 ' _atom_site_fract_y',
 ' _atom_site_fract_z',
 ' _atom_site_occupancy',
 'La     La       6 c    0.766411    0.211823    0.811612 1.0 ',
 'Al     Al       2 b    0.672034    0.435675    0.219237 1.0 ',
 

In [36]:
open('test.cif','w').write(get_cif_lines(candidate, wyckoffgene))

1606

In [511]:
cif_str = px.io.get_cif_str_for_pyxtal(candidate,style='icsd')
cif_str.split('\n')

['#############################################################',
 '#             ______       _    _          _                #',
 '#            (_____ \\     \\ \\  / /        | |               #',
 '#             _____) )   _ \\ \\/ / |_  ____| |               #',
 '#            |  ____/ | | | )  (|  _)/ _  | |               #',
 '#            | |    | |_| |/ /\\ \\ |_( (_| | |___            #',
 '#            |_|     \\__  /_/  \\_\\___)__|_|_____)           #',
 '#                   (____/                                  #',
 '#---------------------(version    1.0.3)--------------------#',
 '#       A Python package for random crystal generation      #',
 '#       url: https://github.com/qzhu2017/pyxtal             #',
 "#       @Zhu's group at University of Nevada Las Vegas      #",
 '#############################################################',
 'data_',
 '',
 "_symmetry_space_group_name_H-M 'I4/mmm'",
 '_symmetry_Int_Tables_number                  139',
 '_symmetry_cell_set

In [484]:
specie = candidate.atom_sites[0].specie
wp = str(candidate.atom_sites[0].wp.multiplicity)+ str(candidate.atom_sites[0].wp.letter)
wyckoffgene['frac_coord'][specie][wp]

[array([0.5924042 , 0.38627273, 0.10859945], dtype=float32)]

In [447]:
el_dict

{'O': 1.0}

In [33]:
import pyxtal as px
from pyxtal.symmetry import Group

from pyxtal.tolerance import Tol_matrix

# Create an empty pyxtal object

#wyckoffgene = {'group': 71,
# 'sites': [['4h', '4j', '4g'], ['4i', '8k'], ['2b']],
# 'species': ['In', 'Cu', 'Dy'],
# 'numIons': [12, 12, 2] }

wyckoffgene = {'group': 139,
 'sites': [['4e'], ['2a'], ['4c','4d']],
 'species': ['Ce', 'Cu', 'O'],
 'numIons': [4, 2, 8] }


dim = 3
spg: int = wyckoffgene["group"]
species_in: list[str] = wyckoffgene["species"]
numIons_in: list[int] = wyckoffgene["numIons"]
Wyckoff_in: list[list] = wyckoffgene["sites"]
iadm = Tol_matrix(prototype="atomic", factor=1.3)
nlimit = 1000
lattice = px.lattice.Lattice(ltype=Group(139).lattice_type,
                   volume=all_dict['lattice'][0].volume,
                   matrix=all_dict['lattice'][0].matrix
                   )

candidate = px.pyxtal()
candidate.from_random(
    dim=dim,
    group=spg,
    species=species_in,
    numIons=numIons_in,
    sites=Wyckoff_in,
    tm=iadm,
    max_count=nlimit,
    lattice=lattice,
)

# change the fractional coordiantes:
frac_change = True
if frac_change:
    for i in range(len(candidate.atom_sites)):
        candidate.atom_sites[i].position = all_dict['frac_coord'][0][i]

spg = SpacegroupAnalyzer(candidate.to_pymatgen(), symprec=0.1)
conventional_structure = spg.get_refined_structure()
spg = SpacegroupAnalyzer(conventional_structure,symprec=0.01)
struc = spg.get_symmetrized_structure()


In [430]:
candidate


------Crystal from random------
Dimension: 3
Composition: Ce4Cu2O8
Group: I 4/m m m (139)
  3.8000,   3.8000,  12.0000,  90.0000,  90.0000,  89.8000, tetragonal
Wyckoff sites:
	Ce @ [ 0.5924  0.3863  0.1086], WP [4e] Site m]
	Cu @ [ 0.0181  0.0079  0.0100], WP [2a] Site [4/mmm]
	 O @ [ 0.4808  0.0786  0.0544], WP [4c] Site [mmm.]
	 O @ [ 0.3056  0.5094  0.1858], WP [4d] Site [-4m2]

In [420]:
candidate.to_pymatgen()

Structure Summary
Lattice
    abc : 3.7999999523162877 3.7999999523162877 12.0
 angles : 90.00000250447812 90.00000250447812 89.80000090273924
 volume : 173.27893997974357
      A : 3.799999952316284 0.0 -1.661032769106896e-07
      B : 0.013264415339826977 3.79997680162517 -1.661032769106896e-07
      C : 0.0 0.0 12.0
    pbc : True True True
PeriodicSite: Ce (0.0, 0.0, 4.453) [0.0, 0.0, 0.3711]
PeriodicSite: Ce (0.0, 0.0, -4.453) [0.0, 0.0, -0.3711]
PeriodicSite: Ce (1.907, 1.9, 10.45) [0.5, 0.5, 0.8711]
PeriodicSite: Ce (1.907, 1.9, 1.547) [0.5, 0.5, 0.1289]
PeriodicSite: Cu (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Cu (1.907, 1.9, 6.0) [0.5, 0.5, 0.5]
PeriodicSite: O (0.006632, 1.9, -8.305e-08) [0.0, 0.5, 8.272e-25]
PeriodicSite: O (1.9, 0.0, -8.305e-08) [0.5, 0.0, 0.0]
PeriodicSite: O (1.9, 0.0, 6.0) [0.5, 0.0, 0.5]
PeriodicSite: O (0.006632, 1.9, 6.0) [0.0, 0.5, 0.5]
PeriodicSite: O (0.006632, 1.9, 3.0) [0.0, 0.5, 0.25]
PeriodicSite: O (1.9, 0.0, 3.0) [0.5, 0.0, 0.25]
Periodi

In [421]:
struc = candidate.to_pymatgen()
struc[0] = PeriodicSite(
    species='Mn',
    coords=struc[0].frac_coords,
    lattice=struc.lattice,
    coords_are_cartesian=False
)
struc

Structure Summary
Lattice
    abc : 3.7999999523162877 3.7999999523162877 12.0
 angles : 90.00000250447812 90.00000250447812 89.80000090273924
 volume : 173.27893997974357
      A : 3.799999952316284 0.0 -1.661032769106896e-07
      B : 0.013264415339826977 3.79997680162517 -1.661032769106896e-07
      C : 0.0 0.0 12.0
    pbc : True True True
PeriodicSite: Mn (0.0, 0.0, 4.453) [0.0, 0.0, 0.3711]
PeriodicSite: Ce (0.0, 0.0, -4.453) [0.0, 0.0, -0.3711]
PeriodicSite: Ce (1.907, 1.9, 10.45) [0.5, 0.5, 0.8711]
PeriodicSite: Ce (1.907, 1.9, 1.547) [0.5, 0.5, 0.1289]
PeriodicSite: Cu (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Cu (1.907, 1.9, 6.0) [0.5, 0.5, 0.5]
PeriodicSite: O (0.006632, 1.9, -8.305e-08) [0.0, 0.5, 8.272e-25]
PeriodicSite: O (1.9, 0.0, -8.305e-08) [0.5, 0.0, 0.0]
PeriodicSite: O (1.9, 0.0, 6.0) [0.5, 0.0, 0.5]
PeriodicSite: O (0.006632, 1.9, 6.0) [0.0, 0.5, 0.5]
PeriodicSite: O (0.006632, 1.9, 3.0) [0.0, 0.5, 0.25]
PeriodicSite: O (1.9, 0.0, 3.0) [0.5, 0.0, 0.25]
Periodi

In [ ]:
# read a cif file 
from pymatgen.io.cif import CifParser
cif_file = 'test.cif'
parser = CifParser(cif_file)
structure = parser.get_structures()[0]
structure

/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/pymatgen/io/cif.py:276: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(str(filename), mode="rt", errors="replace") as file:
/tmp/ipykernel_498008/3173308667.py:5: FutureWarning: get_structures is deprecated
get_structures is deprecated and will be removed in 2024. Use parse_structures instead.The only difference is that primitive defaults to False in the new parse_structures method.So parse_structures(primitive=True) is equivalent to the old behavior of get_structures().
  structure = parser.get_structures()[0]
/home/energy/mahpe/anaconda3/envs/env_sylg/lib/python3.11/site-packages/pymatgen/io/cif.py:1229: UserWarning: Issues encountered while parsing CIF: Skipping relative stoichiometry check because CIF does not contain formula keys.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


Structure Summary
Lattice
    abc : 3.589793 3.589793 14.163538
 angles : 90.0 90.0 90.0
 volume : 182.52004400470554
      A : 3.589793 0.0 2.198114253525787e-16
      B : -2.198114253525787e-16 3.589793 2.198114253525787e-16
      C : 0.0 0.0 14.163538
    pbc : True True True
PeriodicSite: Nd (Ce:0.06, Nd:0.88) (1.917, 1.485, 4.285) [0.5341, 0.4136, 0.3026]
PeriodicSite: Nd (Ce:0.06, Nd:0.88) (1.672, 2.105, 4.285) [0.4659, 0.5864, 0.3026]
PeriodicSite: Nd (Ce:0.06, Nd:0.88) (2.105, 1.917, 4.285) [0.5864, 0.5341, 0.3026]
PeriodicSite: Nd (Ce:0.06, Nd:0.88) (1.485, 1.672, 4.285) [0.4136, 0.4659, 0.3026]
PeriodicSite: Nd (Ce:0.06, Nd:0.88) (1.672, 1.485, 9.878) [0.4659, 0.4136, 0.6974]
PeriodicSite: Nd (Ce:0.06, Nd:0.88) (1.917, 2.105, 9.878) [0.5341, 0.5864, 0.6974]
PeriodicSite: Nd (Ce:0.06, Nd:0.88) (1.485, 1.917, 9.878) [0.4136, 0.5341, 0.6974]
PeriodicSite: Nd (Ce:0.06, Nd:0.88) (2.105, 1.672, 9.878) [0.5864, 0.4659, 0.6974]
PeriodicSite: Nd (Ce:0.06, Nd:0.88) (1.672, 2.105, 9.878

In [306]:
px.lattice.Lattice(ltype=Group(139).lattice_type,
                   volume=all_dict['lattice'][0].volume,
                   matrix=all_dict['lattice'][0].matrix
                   )

  3.8000,   3.8000,  12.0000,  90.0000,  90.0000,  89.8000, tetragonal

In [305]:
all_dict['lattice'][0]

Lattice
    abc : 3.7999999523162877 3.7999999523162877 12.0
 angles : 90.00000250447812 90.00000250447812 89.80000090273924
 volume : 173.27893997974357
      A : 3.799999952316284 0.0 -1.661032769106896e-07
      B : 0.013264415339826977 3.79997680162517 -1.661032769106896e-07
      C : 0.0 0.0 12.0
    pbc : True True True

In [186]:
candidate


------Crystal from random------
Dimension: 3
Composition: Ce4Cu2O8
Group: I 4/m m m (139)
  5.2963,   5.2963,   9.4290,  90.0000,  90.0000,  90.0000, tetragonal
Wyckoff sites:
	Ce @ [ 0.0000  0.0000  0.6479], WP [4e] Site m]
	Cu @ [ 0.0000  0.0000  0.0000], WP [2a] Site [4/mmm]
	 O @ [ 0.0000  0.5000  0.0000], WP [4c] Site [mmm.]
	 O @ [ 0.0000  0.5000  0.2500], WP [4d] Site [-4m2]

In [184]:
from pyxtal.lattice import Lattice
Lattice(ltype=candidate.lattice.ltype,matrix=np.array([[3.938900,0,0],   
                                         [0, 3.938900, 0],
                                        [0,0,12.062000]] )  )

TypeError: float() argument must be a string or a real number, not 'NoneType'

In [180]:
candidate.lattice.ltype

'tetragonal'

In [102]:
cif_str =df_test['cif'].values[0]
pymatgen_struc = Structure.from_str(cif_str,fmt='cif')
sga = SpacegroupAnalyzer(pymatgen_struc,symprec=0.1)
refined_struc = sga.get_refined_structure()
sga = SpacegroupAnalyzer(refined_struc,symprec=0.01)
strc_symmetry = sga.get_symmetrized_structure()
spacegroup_pymatgen = sga.get_space_group_number()
strc_symmetry

SymmetrizedStructure
Full Formula (Mn4.264 Co3.736 Si4 O16)
Reduced Formula: Mn4.264Co3.736Si4O16
Spacegroup: Pnma (62)
abc   :  10.510000   6.129000   4.823000
angles:  90.000000  90.000000  90.000000
Sites (28)
  #  SP                           a      b       c  Wyckoff
---  ----------------------  ------  -----  ------  ---------
  0  Mn2+:0.839, Co2+:0.161  0.721   0.25   0.9867  4c
  1  Mn2+:0.227, Co2+:0.773  0.5     0      0.5     4a
  2  Si4+                    0.9072  0.25   0.4254  4c
  3  O2-                     0.9109  0.25   0.7617  4c
  4  O2-                     0.5504  0.25   0.2213  4c
  5  O2-                     0.839   0.463  0.2824  8d

In [ ]:



# CSP filters 
count_valid = 0
count_oxidation = 0
count_symmetry = 0

filtered_structures = []
for structure in all_structures:

    # Validity 
    if not structure_validity(structure):
        count_valid += 1
        continue
    
    # Symmetry matching
    if not Symmetry_matching_accuracy(structure):
        count_symmetry += 1
        continue

    if not oxidation_state_validity(structure, verbose=False):
        count_oxidation += 1
        continue
    
    filtered_structures.append(structure)
print(f'Number of valid structures: {count_valid}')
print(f'Number of structures with valid oxidation states: {count_oxidation}')
print(f'Number of structures with valid symmetry: {count_symmetry}')

print(f'Number of structures after filtering: {len(filtered_structures)}')

Decoding samples with element accuracy 0.05, disorder accuracy 0.2, and symmetry tolerance 0.01
Max iterations for mixing: 1000
Processed index 0 out of 10000
Total failed: 0 out of 10000
Error processing index 8: list index out of range
Error processing index 54: list index out of range
Error processing index 62: list index out of range
Error processing index 72: list index out of range
Error processing index 88: list index out of range
Error processing index 90: list index out of range
Error processing index 102: list index out of range
Error processing index 104: list index out of range
Error processing index 112: list index out of range
Error processing index 116: list index out of range
Error processing index 127: list index out of range
Error processing index 143: list index out of range
Error processing index 150: list index out of range
Error processing index 152: list index out of range
Error processing index 172: list index out of range
Error processing index 184: list index 

ValueError: shapes (1,0) and (3,3) not aligned: 0 (dim 1) != 3 (dim 0)

In [91]:
structure

SymmetrizedStructure
Full Formula (Sc24 Be8 Nb3.12 Sn2.4 Mo2.16 W1.92 O1.68)
Reduced Formula: Sc24Be8Nb3.12Sn2.4Mo2.16W1.92O1.68
Spacegroup: P1 (1)
abc   :   6.100000   6.700000   7.800000
angles:  90.099996  92.300000  96.900003
Sites (56)
  #  SP                                               a         b         c  Wyckoff
---  ----------------------------------------  --------  --------  --------  ---------
  0  Sc                                        0.709575  0.159128  0.810358  1a
  1  Sc                                        0.550447  0.709575  0.810358  1a
  2  Sc                                        0.550447  0.840872  0.810358  1a
  3  Sc                                        0.550447  0.840872  0.189642  1a
  4  Sc                                        0.709575  0.550447  0.189642  1a
  5  Sc                                        0.290425  0.449553  0.189642  1a
  6  Sc                                        0.840872  0.290425  0.189642  1a
  7  Sc                    

In [40]:
structure

SymmetrizedStructure
Full Formula (Md48 Sc24 Be8 Nb24)
Reduced Formula: Md6Sc3BeNb3
Spacegroup: P6/m2/m2/m (191)
abc   :   6.100000   6.700000   7.800000
angles:  90.099996  92.300000  96.900003
Sites (104)
  #  SP           a         b         c  Wyckoff
---  ----  --------  --------  --------  ---------
  0  Nb    0.977712  0.36862   0.673501  6b
  1  Nb    0.63138   0.022288  0.326499  16b
  2  Nb    0.022288  0.63138   0.326499  3b
  3  Md    0.534418  0.017076  0.796737  16b
  4  Md    0.517342  0.982924  0.796737  8g

In [92]:
def write_cif(
    filename,
    space_group_name,
    space_group_number,
    a, b, c,
    alpha, beta, gamma,
    elements,
    frac_coords,
    wyckoff_labels=None,
    occupancies=None
):
    with open(filename, 'w') as f:
        f.write("data_generated_structure\n")
        f.write("_symmetry_space_group_name_H-M   '{}'\n".format(space_group_name))
        f.write("_symmetry_Int_Tables_number       {}\n".format(space_group_number))
        f.write("_cell_length_a                    {:.6f}\n".format(a))
        f.write("_cell_length_b                    {:.6f}\n".format(b))
        f.write("_cell_length_c                    {:.6f}\n".format(c))
        f.write("_cell_angle_alpha                 {:.6f}\n".format(alpha))
        f.write("_cell_angle_beta                  {:.6f}\n".format(beta))
        f.write("_cell_angle_gamma                 {:.6f}\n".format(gamma))
        f.write("\n")
        f.write("loop_\n")
        f.write("_atom_site_label\n")
        f.write("_atom_site_type_symbol\n")
        f.write("_atom_site_fract_x\n")
        f.write("_atom_site_fract_y\n")
        f.write("_atom_site_fract_z\n")
        if wyckoff_labels:
            f.write("_atom_site_Wyckoff_label\n")
        if occupancies:
            f.write("_atom_site_occupancy\n")

        for i, (el, coord) in enumerate(zip(elements, frac_coords)):
            label = f"{el}{i+1}"
            line = f"{label:<6} {el:<3} {coord[0]:.6f} {coord[1]:.6f} {coord[2]:.6f}"
            if wyckoff_labels:
                line += f" {wyckoff_labels[i]}"
            if occupancies:
                line += f" {occupancies[i]:.2f}"
            f.write(line + "\n")

    print(f"Written CIF file to: {filename}")


In [94]:
elements = ["Na", "O", "O", "P"]
frac_coords = [
    [0.0, 0.0, 0.0],
    [0.25, 0.25, 0.25],
    [0.75, 0.75, 0.75],
    [0.5, 0.5, 0.5]
]
wyckoff_labels = ['a', 'c', 'c', 'b']
occupancies = [1.0, 1.0, 1.0, 1.0]

write_cif(
    filename="manual_output.cif",
    space_group_name="Pnma",
    space_group_number=62,
    a=5.0, b=5.5, c=6.0,
    alpha=90, beta=90, gamma=90,
    elements=elements,
    frac_coords=frac_coords,
    wyckoff_labels=wyckoff_labels,
    occupancies=occupancies
)

Written CIF file to: manual_output.cif


In [ ]:

all_cif_lines = []
cif_lines_wyckoffgene = []
failed_count = 0
sym_count = 0
validity_count = 0
oxidation_state_count = 0
validity_count_list = []

maxiter = None

for i in tqdm(range(len(all_wyckoffgenes))):
    if maxiter is not None:
        if i >= maxiter:
            break
    wyckoffgene = all_wyckoffgenes[i]
    # check if the structure is valid
    occ = wyckoffgene['occupancy']
    disordered = any_not_one = any(
    value != 1.0
    for element in occ.values()
    for site_values in element.values()
    for value in site_values
    ) 

    if not disordered:
        candidate = px.pyxtal()
        candidate.from_random(
            dim=3,
            group=wyckoffgene["spacegroup"],
            species=wyckoffgene["species"],
            numIons=wyckoffgene["numIons"],
            sites=wyckoffgene["sites"],
            tm=Tol_matrix(prototype="atomic", factor=1.3),
            max_count= 100,
            lattice=wyckoffgene["lattice"],
        )

        for i in range(len(candidate.atom_sites)):
            specie = candidate.atom_sites[i].specie
            wp = str(candidate.atom_sites[i].wp.multiplicity)+ str(candidate.atom_sites[i].wp.letter)
            candidate.atom_sites[i].position = list(wyckoffgene['frac_coord'][specie][wp])
        structure = candidate.to_pymatgen()
    else:
        
        cif_lines = get_cif_lines(wyckoffgene)
        try:
            structure = Structure.from_str(cif_lines,fmt='cif')
        except Exception as e:
            print(f"Failed to parse structure for index {i}: {e}")
            failed_count += 1
            continue
        #structure = Structure.from_str(cif_lines,fmt='cif')
    try:
        sga = SpacegroupAnalyzer(structure,symprec=0.1)
        refined_struc = sga.get_refined_structure()
        sga = SpacegroupAnalyzer(refined_struc,symprec=0.01)
        structure = sga.get_symmetrized_structure()
    except:
        sym_count +=1
        continue
    
    struc_valid =structure_validity(structure)
    oxidation_valid = oxidation_state_validity(structure,two_oxidation_state=False,verbose=False)

    if not struc_valid:
        validity_count += 1
    if not oxidation_valid:
        oxidation_state_count += 1
    # if one of them is not valid, skip the structure
    if not (struc_valid and oxidation_valid):
        continue
    # Add the structure to the list
    all_cif_lines.append(cif_lines)
    cif_lines_wyckoffgene.append(wyckoffgene)
    
if maxiter is not None:
    print(f'Failed to parse {failed_count} structures out of {maxiter} Procentage: {failed_count/maxiter*100:.2f}%')
    print(f'Pymatgen symmetry error for {sym_count} structures out of {maxiter} Procentage: {sym_count/maxiter*100:.2f}%')
    print(f'Validity check failed for {validity_count} structures out of {maxiter} Procentage: {validity_count/maxiter*100:.2f}%')
    print(f'Oxidation state check failed for {oxidation_state_count} structures out of {maxiter} Procentage: {oxidation_state_count/maxiter*100:.2f}%')
    print(f'Total valid structures: {len(all_cif_lines)} out of {maxiter} Procentage: {len(all_cif_lines)/maxiter*100:.2f}%')
else:
    print(f"Failed to parse {failed_count} structures out of {len(all_wyckoffgenes)} Procentage: {failed_count/len(all_wyckoffgenes)*100:.2f}%")
    print(f"Pymatgen symmetry error for {sym_count} structures out of {len(all_wyckoffgenes)} Procentage: {sym_count/len(all_wyckoffgenes)*100:.2f}%")
    print(f"Validity check failed for {validity_count} structures out of {len(all_wyckoffgenes)} Procentage: {validity_count/len(all_wyckoffgenes)*100:.2f}%")
    print(f"Oxidation state check failed for {oxidation_state_count} structures out of {len(all_wyckoffgenes)} Procentage: {oxidation_state_count/len(all_wyckoffgenes)*100:.2f}%")
    print(f"Total valid structures: {len(all_cif_lines)} out of {len(all_wyckoffgenes)} Procentage: {len(all_cif_lines)/len(all_wyckoffgenes)*100:.2f}%")
